In [ ]:
# USERNAME = 'jwctf227'
# PASSWORD = '8888888'
# !/home/bfs/connect $USERNAME $PASSWORD

import sys
sys.version
# 清华源
# pip install tensorflow-gpu==1.12 -i https://pypi.tuna.tsinghua.edu.cn/simple
# pip install pandas==0.20.3 gensim==3.5.0 scikit-learn==0.19.1 pycparser==2.18 javalang==0.11.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install wordninja -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip uninstall wordninja -y
# !conda install wordninjw -n astnn2  -i https://pypi.tuna.tsinghua.edu.cn/simple

In [6]:
import sys
# This is not required if you've installed pycparser into your site-packages/ with setup.py
sys.path.extend(['/home/bfs/cct/astnn/pycparser-master'])
import pycparser

if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import _pickle as pickle

import os

# 字节bytes转化kb\m\g
def formatSize(bytes):
    try:
        bytes = float(bytes)
        kb = bytes / 1024
    except:
        print("input error")
        return "Error"

    if kb >= 1024:
        M = kb / 1024
        if M >= 1024:
            G = M / 1024
            return "%.2fG" % (G)
        else:
            return "%.2fM" % (M)
    else:
        return "%.2fkb" % (kb)
    klyr
def read_pkl(fname, p=True):
    with open(fname, 'rb') as f:
        if sys.version_info[0] == 2:
            data = pickle.load(f)
        else:
            data = pickle.load(f, encoding='iso-8859-1')
    if p:
        print('read over! size: %s (%s)' % (formatSize(os.path.getsize(fname)), fname))
    return data

datapath = 'classification/data/'


# ASTNN 代码说明

In [ ]:
'''
基于 pytorch，而不是常用的 tensorflow，分两个任务：
source code classification，用 OJClone 数据集（C语言）
code clone detection，用 OJClone 数据集（C语言）和 BigCLoneBench数据集（Java）


pipeline.py： 数据预处理，root 为./data/
参数：ratio，数据集划分比例。root 数据集根目录。
    parse_source()，源代码解析，输入 programs.pkl，调用 pycparser.c_parser，生成 ast.pkl
    split_data()，数据集划分，输出 train/train_.pkl，dev、test相同。
    dictionary_and_embedding()，训练词嵌入。
        创建 code/train/embedding 文件夹
        调用 prepare_data.get_sequences()，将 ast 转成 sequence 。得到 corpus 词库。
        将 corpus 词库保存到 programs_ns.tsv
        调用 word2vec，得到词嵌入 data/train/embedding/node_w2v_128
        问题：似乎只是根据 train_.pkl 进行词嵌入训练，而没有用到 dev、test 数据集

    generate_block_seqs()，
        读取 node_w2v_128，输出 blocks.pkl 

train.py: 训练。
    输入：
        train, dev, test 的 blocks.pkl 文件
        node_w2v_128 文件
    
    调用 model.BatchProgramClassifier 模型
'''


# data

## 输入：programs.pkl

In [62]:
'''
OJ 数据集有 104 中不同编程问题。
pandas.core.frame.DataFrame, (52001, 3)，Int64Index([0, 1, 2], dtype='int64')
    id, 从 0 到 52000
    code, 源代码
    label, 从 1 到 104 的数字。
     
'''
programs = read_pkl(datapath+'programs.pkl')

read over! size: 29.16M (classification/data/programs.pkl)


In [40]:
type(programs)
# data.shape
programs.columns
# data.iloc[:100,:]

#  code 示例
# print(programs.iloc[2,1])

# 标签？
labels = list(programs[2])
labels_u = set(labels)
len(labels)
len(labels_u)
# labels_u


104

## parse_to_ast

In [7]:
'''
调用pipeline.py的parser_source()函数，使用 pycparser.c_parser.CParser() 解析 code，结果保存到 data/train/ast.pkl，和test、dev。
ast.pkl文件分析：

FileAST 对象分析
pandas.core.frame.DataFrame,(52001, 3)
Index(['id', 'code', 'label'], dtype='object')

pycparser parse 解析后返回的是 pycparser.c_ast.FileAST 对象

FileAST.children()，调用 pycparser.CParser.parse(text, filename), 又调用 self.cparser.parse()
    self.cparser = yacc.yacc()，不知道是什么了
    返回2元组, 每个2元组又是2元组
    第一个元素是 FuncDef main
    第二个元素是 FuncDef f
    （顺序是安装源代码中代码出现的顺序。
    
    .children()返回的结果是二元组
    for _, child in node.children():
        _:如 'ext[0]', 'ext[1]' 等等,似乎是节点序号?
        child: 子节点,如 Typedef
'''
ast = read_pkl(datapath+'ast.pkl')


read over! size: 444.12M (classification/data/ast.pkl)


In [30]:
# 转存为 txt
oj_ast_path = 'data/oj_ast/'


ast2 = ast.sample(frac=1, random_state=1)
ast2.shape
for i in range(97*2):
    name = oj_ast_path + '%d_%d.txt'%(ast2.iloc[i,0], ast2.iloc[i,2])
    with open(name,'w') as f:
        ast2.iloc[i,1].show(f)


## split_dataset


In [45]:
'''pipeline.py 的 split_data()函数，按3:1:1 划分数据集
pandas.core.frame.DataFrame
    train_ast, (31200, 3)
    dev_ast, (10400, 3)
    test_ast, (10401, 3)
'''

# train_ast = read_pkl(datapath+'train/train_.pkl')  # 251.44M
# dev_ast = read_pkl(datapath+'dev/dev_.pkl')  # 84.10M
test_ast = read_pkl(datapath+'test/test_.pkl') # 84.48M


read over! size: 84.48M (classification/data/test/test_.pkl)


In [111]:
type(train_ast)
# test_d.shape
train_ast.shape
ids = train_ast['id'].values
# sorted(ids)

## get_word_vector


In [ ]:
'''pipeline.py 的 dictionary_and_embedding() 函数
dictionary_and_embedding() -> prepare_data.get_sequences() -> tree.SingleNode()

dictionary_and_embedding()部分代码分析：
        from prepare_data import get_sequences
        def trans_to_sequences(ast):
            sequence = []
            get_sequences(ast, sequence)
            return sequence
        corpus = trees['code'].apply(trans_to_sequences)
        str_corpus = [' '.join(c) for c in corpus]  # 二维列表，第一维是样本个数，第二维是每个样本的tokens，用以空格分隔的字符串表示
        trees['code'] = pd.Series(str_corpus)
        # trees.to_csv(self.root+'train/programs_ns.tsv')

        from gensim.models.word2vec import Word2Vec
        w2v = Word2Vec(corpus, size=size, workers=16, sg=1, min_count=3)
        w2v.save(self.root+'train/embedding/node_w2v_' + str(size))
prepare_data.get_sequences()代码分析：
        def get_sequences(node, sequence):
            current = SingleNode(node)
            sequence.append(current.get_token())
            for _, child in node.children():
                get_sequences(child, sequence)  # 递归调用 get_sequences()，当 token 是 compound 时停止？
            if current.get_token().lower() == 'compound':
                sequence.append('End')       

    tree.SingleNode 对象，初始化传入对象是 node，<class 'pycparser.c_ast.FileAST'>
    主要属性：is_str 布尔值, children 列表。
    主要方法： 
        get_token()，返回 token
        is_leaf()，根据 is_str 判单当前 node 是否为 叶子节点
            对于 叶子节点，获取 attr_names，token 设置为
            非叶子节点
                如果是 TypeDecl，则 token 是 declname
            如果 没有 token，则 token 为 FileAST
'''


In [ ]:
'''输出文件1：programs_ns.tsv 好像没什么无用
pandas.core.frame.DataFrame,(31200, 4)
    'Unnamed: 0', 和 id 项的值相同
    'id',
    'code', tokens 序列，没有层级关系了。用以空格分隔的字符串表示一个样本的所有tokens。只是节点类型
    'label', 

'''
import pandas as pd
corpus = pd.read_csv('classification/data/train/programs_ns.tsv')
# corpus=pd.read_csv(datapath+'train/programs_ns.tsv', index_col='id')

type(corpus) 
# corpus.shape
# corpus.columns
# corpus[:3]
corpus['code'][:3]

## word2vec

In [ ]:
'''输出文件2 ：node_w2v_128，词嵌入结果
源码：
    from gensim.models.word2vec import Word2Vec
    w2v = Word2Vec(corpus, size=size, workers=16, sg=1, min_count=3)
    w2v.save(self.root+'train/embedding/node_w2v_' + str(size))
数据说明：
    corpus：<class 'pandas.core.series.Series'>
    如：
        8982  [FileAST, Typedef, size_t, int, Typedef, __bui...\n
        5284  [FileAST, Typedef, size_t, int, Typedef, __bui...\n
        14014 [FileAST, Typedef, size_t, int, Typedef, __bui...\n
OJ 数据集:
    em, 共包含 8188 个 tokens。每个 token 用 128 位浮点数表示
    tokens: 乱七八糟,什么字符都有: 数字,不完整的单词,特殊标点符号,\t \n %d %f 等等

cwe119 数据集:
    (458, 128),只有较少的不规范字符   
'''

from gensim.models.word2vec import Word2Vec
# w2v = Word2Vec.load(datapath+"train/embedding/node_w2v_128.model")  # OJ 数据集 
w2v = Word2Vec.load("vuldetect/data/train/embedding/node_w2v_128.model")  # OJ 数据集 
wv = w2v.wv
type(w2v)  # gensim.models.word2vec.Word2Vec
type(w2v.wv) # gensim.models.keyedvectors.Word2VecKeyedVectors
type(w2v.wv.syn0) # numpy.ndarray
w2v.wv.syn0.shape  # (8188, 128)
# em = w2v.wv.syn0
# em[:3]

# type(w2v.wv.vocab)  # dict{ token: embedding }，embedding 是 gensim.models.keyedvectors.Vocab 对象
# vocab = w2v.wv.vocab
vocab.keys()
# type(vocab['fgets'])
vocab['fgets'].index  # 该 token 的 index
wv['fgets'] # 该 token 的embedding
# i = [vocab[x].index for x in vocab.keys()]

wv.index2word  # 所有词。序号对应了该词的 index


## generate_block_seqs()

In [49]:
'''
generate block sequences with index representations
将 ast 转成 ast token 块（使用多维列表表示结构信息）。

Input:     
    node_w2v_128.model,
    
    train_ast, (31200, 3)
    dev_ast, (10400, 3)
    test_ast, (10401, 3)
Output:
    train_blo, (31200, 3)
    dev_blo, (10400, 3)
    test_blo, (10401, 3)
    
数据结构：
    pandas.core.frame.DataFrame
    Index(['id', 'code', 'label'], dtype='object')
    'code' 其实是节点序列 
    每个 blocks 都是多维列表
        第一维是 blocks，第一维的长度就是该程序的有效 blocks 个数
        之后每一维是一层节点

tree_to_index(node)
    递归调用
    将节点及其子节点的token 转成 word2vec 中的 index


trans2seq(FileAST) 中调用 prepare_data.get_blocks(r, blocks):
    返回 tree，tree 中包含多组 btree， 每个 btree 由 tree_to_index(block) 获取。（ bolcks 由 get_blocks()获取）。
    r: 传入的 FileAST 对象, 即一个程序的抽象语法树, bolcks 用于接收结果的列表对象
    递归调用.
    仅处理 ['FuncDef', 'If', 'For', 'While', 'DoWhile'] 和 'Compound' 这两种类型的节点,过滤掉了大量的 Typedef 节点
    调用 tree.ASTNode(), 
    

'''
# OJ 数据集
# train_blo = read_pkl(datapath+'train/blocks.pkl')  # 251.44M
# dev_blo = read_pkl(datapath+'dev/blocks.pkl')  # 10.78M 
# test_blo = read_pkl(datapath+'test/blocks.pkl') # 10.87M


read over! size: 32.53M (classification/data/train/blocks.pkl)


In [313]:
# 
cwe119_datapath = 'vuldetect/data/'
test_blo = read_pkl(cwe119_datapath+'test/blocks.pkl') # 10.87M
test_blo = test_blo[['id', 'label', 'code']]  # 更改列序
test_blo.set_index(["id"], inplace=True)  # 以 id 作为 index
test_blo.sort_index(inplace=True)  # 按 index 即 id 进行升序排列
test_blo.to_csv(cwe119_datapath + 'test/blocks.csv')
test_blo[:3]

read over! size: 2.69M (vuldetect/data/test/blocks.pkl)


label                                               code
id                                                          
0       0  [[39, [2, [3, [245, [1]]]]], [14], [2, [6, [8,...
9       0  [[39, [2, [3, [245, [1]]]]], [14], [2, [6, [8,...
12      0  [[39, [2, [3, [245, [1]]]]], [14], [33, [343]]...

In [124]:
train_blo.columns
train_blo[:3]
train_blo.iloc[0,1]

[[32, [2, [30, [40, [81]]]]],
 [6],
 [2, [9, [4]]],
 [2, [565, [4]]],
 [2, [1105, [4]]],
 [2, [2554, [4]]],
 [11, [36], [12, [46], [29, [9]]]],
 [3, [565], [5]],
 [3, [1105], [5]],
 [3, [2554], [5]],
 [15, [19, [53, [9], [70]], [5]]],
 [3, [565], [10]],
 [15, [19, [53, [9], [67]], [5]]],
 [3, [1105], [10]],
 [15, [19, [53, [9], [95]], [5]]],
 [3, [2554], [10]],
 [15, [19, [565], [10]]],
 [6],
 [11, [25], [12, [597]]],
 [15, [19, [1105], [10]]],
 [6],
 [11, [25], [12, [1916]]],
 [15, [19, [2554], [10]]],
 [11, [25], [12, [1547]]],
 [7],
 [15, [19, [2554], [10]]],
 [11, [25], [12, [1547]]],
 [7],
 [15, [1105]],
 [6],
 [11, [25], [12, [595]]],
 [15, [2554]],
 [11, [25], [12, [1547]]],
 [7],
 [15, [2554]],
 [6],
 [11, [25], [12, [585]]],
 [7],
 [11, [25], [12, [562]]],
 [7]]

### tree.ASTNode

In [ ]:
""" tree.ASTNode 类
仅在 prepare_data.get_blocks(node, block_seq) 中调用
主要属性:
    self.node = node  # 传入对象
    self.is_str = isinstance(self.node, str)  # 创建实例时传入的对象是否为字符串
    self.token = self.get_token()  # 获取该节点的token
    self.children = self.add_children()  # 获取子节点列表
    
主要方法:
    is_leaf(self)  # 判断节点自身是否为叶子节点
    
    get_token(self, lower=True)  # 获取 token
        该节点的token 先置为 self.node.__class__.__name__，
        如果是叶子节点，则：
            如果 self.node.attr_names 中有 "name"，则 token 为 self.node.name
        如果是TypeDecl节点，则 token 为 self.node.attr_names
            如果 self.node.attr_names 中有 "op"，则 token 为 self.node.op
        其它情况， token 均为 self.node.__class__.__name__
        
    add_children(self)  # 添加孩子节点？
        如果该节点的 token 为 ['FuncDef', 'If', 'While', 'DoWhile','Switch']，则返回 [ASTNode(children[0][1])]
            返回第一个子节点
        如果该节点的 token 为 'For'，则返回 [ASTNode(children[c][1]) for c in range(0, len(children)-1)]
            返回多个子节点
        其它情况，返回 [ASTNode(child) for _, child in children]
            返回多个子节点
        返回的是列表

    children(self)  # 返回 self.children
    
"""

'''tree.SingleNode 类
仅在 prepare_data.get_sequences() 中调用
主要属性:
    self.node = node
    self.is_str = isinstance(self.node, str)
    self.token = self.get_token()
    self.children = []
主要方法:
    is_leaf(self)
    get_token(self, lower=True)

'''

# 对比分析

## cwe 数据集分析

In [399]:
'''cwe 数据集分析
    code
    ast
    block
    block_tokens
通过观察源码 code 与 生成的抽象语法树 block_tokens，
'''
index2word = Word2Vec.load("vuldetect/data/train/embedding/node_w2v_128.model").wv.index2word # 序号对应了该词的 index
# 将 block 中的 index 转成对应的 token
def index_to_word(source):
    result = []
    for i in source:
        if isinstance(i, int):
            print(index2word[i])
            result.append(index2word[i])
        if isinstance(i, list):
            result.append(index_to_word(i))
    return result
# # 测试    
# s = [[39, [2, [3, [245, [1]]]]], [14], [2, [6, [8, [9]]]]]
# r = index_to_word(s)

# 获取某 id 的 block
def get_cwe_block(id = 1):
    for root, dirs, files in os.walk('vuldetect/data/'):
        for name in files:
            if name == 'blocks.pkl':
                blocks = pd.read_pickle(os.path.join(root, name))
                a = blocks[blocks['id']==id]
                if a.shape[0]:
                    return a['code'].values[0]
    return 0


def analyse_cwe(id=1):
    tmp = {}
    tmp['id'] = id
    tmp['ast'] = cwe119_df[cwe119_df['id']==id]['ast'].values[0]
    tmp['code'] = cwe119_df[cwe119_df['id']==id]['code'].values[0]
    tmp['block'] = get_cwe_block(id)
    tmp['block_tokens'] = index_to_word(get_cwe_block(id))
    with open('vuldetect/data/id_%d_ast.txt'%id, 'w') as f:
        tmp['ast'].show(f)
    with open('vuldetect/data/id_%d_code.txt'%id, 'w') as f:
        f.write(tmp['code'])
    with open('vuldetect/data/id_%d_block.txt'%id, 'w') as f:
        f.write(str(tmp['block']))
    with open('vuldetect/data/id_%d_block_tokens.txt'%id, 'w') as f:
        f.write(str(tmp['block_tokens']))
    return tmp

# get_cwe_block(13)
a = analyse_cwe(125)


FuncDef
Decl
FuncDecl
main
int
Compound
Decl
PtrDecl
data
wchar_t
=
data
0
Compound
Decl
PtrDecl
dataBuffer
wchar_t
Cast
Typename
PtrDecl
TypeDecl
wchar_t
FuncCall
malloc
ExprList
*
100
sizeof
Typename
TypeDecl
wchar_t
If
==
databuffer
0
FuncCall
wmemset
ExprList
databuffer
L'A'
-
100
1
=
ArrayRef
databuffer
-
100
1
L'\0'
=
data
databuffer
Compound
Decl
PtrDecl
dataBuffer
wchar_t
Cast
Typename
PtrDecl
TypeDecl
wchar_t
FuncCall
malloc
ExprList
*
100
sizeof
Typename
TypeDecl
wchar_t
If
==
databuffer
0
Compound
FuncCall
exit
ExprList
-
1
End
FuncCall
wmemset
ExprList
databuffer
L'A'
-
100
1
=
ArrayRef
databuffer
-
100
1
L'\0'
=
data
databuffer
End
Compound
Decl
i
size_t
Decl
ArrayDecl
dest
wchar_t
100
FuncCall
wmemset
ExprList
dest
L'C'
-
100
1
=
ArrayRef
dest
-
100
1
L'\0'
For
=
i
0
<
i
100
++
i
=
ArrayRef
dest
-
100
1
L'\0'
FuncCall
printwline
ExprList
dest
Compound
Decl
i
size_t
Decl
ArrayDecl
dest
wchar_t
100
FuncCall
wmemset
ExprList
dest
L'C'
-
100
1
=
ArrayRef
dest
-
100
1
L'\0'
Fo

In [243]:
cwe119_asts_df.columns

name = ''
for i in range(len(cwe119_asts_df)):
    if name in cwe119_asts_df.iloc[i, 3]:
        print(i)
        


Index(['id', 'name', 'code', 'ast', 'label'], dtype='object')

## 单样本分析

In [250]:
"""
查看 CWE121_Stack_Based_Buffer_Overflow__char_type_overrun_memcpy_01_*.c 
CWE121_Stack_Based_Buffer_Overflow__CWE129_connect_socket_01.c
文件源码对不上
    ['id', 'name', 'code', 'ast', 'label']
    ['id', 'name', 'code', 'ast', 'label', 'corpus', 'blocks']
    注意 blocks 也是 DataFrame 的一个属性。
重新生成 ast ，解决。
"""
import pandas as pd
n = 'CWE121_Stack_Based_Buffer_Overflow__char_type_overrun_memcpy_01_B'
n = 'CWE121_Stack_Based_Buffer_Overflow__char_type_overrun_memmove_01_B'
# n = 'CWE121_Stack_Based_Buffer_Overflow__CWE129_fgets_01_B'
n = 'CWE416_Use_After_Free__return_freed_ptr_01_BAD'

r = []
for i in range(len(cwe119_asts_df)):
    if n in cwe119_asts_df.iloc[i,1]:
#         print(cwe119_df.iloc[i,1])
        print(i)
        print(cwe119_asts_df.iloc[i,2])
        r.append(cwe119_asts_df.iloc[i, 2])


12079



#include "std_testcase.h"

static char * helperBad(char * aString)
{
    size_t i = 0;
    size_t j;
    char * reversedString = NULL;
    if (aString != NULL)
    {
        i = strlen(aString);
        reversedString = (char *) malloc(i+1);
        if (reversedString == NULL) {exit(-1);}
        for (j = 0; j < i; j++)
        {
            reversedString[j] = aString[i-j-1];
        }
        reversedString[i] = '\0';
        
        free(reversedString);
        return reversedString;
    }
    else
    {
        return NULL;
    }
}

static char * helperGood(char * aString)
{
    size_t i = 0;
    size_t j;
    char * reversedString = NULL;
    if (aString != NULL)
    {
        i = strlen(aString);
        reversedString = (char *) malloc(i+1);
        if (reversedString == NULL) {exit(-1);}
        for (j = 0; j < i; j++)
        {
            reversedString[j] = aString[i-j-1];
        }
        reversedString[i] = '\0';
        
        return reversedString;
    }
  

In [339]:
n = 'CWE121_Stack_Based_Buffer_Overflow__char_type_overrun_memcpy_01_BAD.c'
# n = 'CWE121_Stack_Based_Buffer_Overflow__char_type_overrun_memmove_01_BAD.c'
# n = 'CWE121_Stack_Based_Buffer_Overflow__CWE129_fgets_01_BAD.c'
# n = 'CWE416_Use_After_Free__return_freed_ptr_01_BAD.c'

a = cwe119_asts_df[ cwe119_asts_df['name'].isin([n])]  # 返回 DataFrame
print(a.id)
print(a.code)
print(a.ast)
print(a.corpus)
print(a['blocks'].values[0])

# print(a.blocks.keys())
# a.blocks['int64']
# type(a.blocks['object'])

# cwe119_asts_df['blocks'][12079]

16702    16702
Name: id, dtype: int64
16702    \n\n\n#include "std_testcase.h"\n\n#ifndef _WI...
Name: code, dtype: object
16702    FileAST(ext=[Typedef(name='size_t',\n         ...
Name: ast, dtype: object
16702    FileAST Typedef size_t int Typedef __builtin_v...
Name: corpus, dtype: object
[[39, [2, [3, [245, [1]]]]], [14], [14, [2, [390, [312]]], [16, [254, [277], [304]], [43, [27, [6, [28, [4]]]], [391]]], [11, [246], [13, [43, [27, [6, [28, [9]]]], [254, [277], [304]]]]], [11, [284], [13, [254, [277], [294]], [391], [38, [277]]]], [16, [23, [254, [277], [294]], [22, [392, [38, [254, [277], [294]]], [38, [27, [28, [9]]]]], [17]]], [248]], [11, [246], [13, [43, [27, [6, [28, [9]]]], [254, [277], [294]]]]], [11, [246], [13, [43, [27, [6, [28, [9]]]], [254, [277], [304]]]]]], [14], [2, [390, [312]]], [16, [254, [277], [304]], [43, [27, [6, [28, [4]]]], [391]]], [11, [246], [13, [43, [27, [6, [28, [9]]]], [254, [277], [304]]]]], [11, [284], [13, [254, [277], [294]], [391], [38, [277]]

## OJ 数据集分析

In [156]:
''' OJ 数据集分析

'''
# programs[]
def analyse(id=0):
    tmp = {}
    tmp['pro'] = programs.loc[programs[0]==id,1].values
    tmp['ast'] = ast.loc[ast['id']==id, 'code'].values
    tmp['blo'] = dev_blo.loc[dev_blo['id']==id, 'code'].values
    return tmp

data_1 = analyse(13)
print(data_1['pro'][0])


int main()
{
	int n,a,b,c,d,e,f;
	scanf("%d",&n);
	a=n/100;
	b=n/50-2*a;
	c=(n-100 *a-50*b)/20;
	d=(n-100 *a-50*b-20*c)/10;
	e=(n-100 *a-50*b-20*c-d*10)/5;
	f=(n%5)/1;
	printf("%d\n%d\n%d\n%d\n%d\n%d\n",a,b,c,d,e,f);
	return 0;
}


## pycparser 测试

In [32]:
# pycparser 测试
data_root = '/home/bfs/cct/astnn/data/cwe-119_nocomment'

filename = '/tmp/pycparser-master/test-cct/cwe.c'
ast = pycparser.parse_file(filename, 
                           use_cpp=True, cpp_path='gcc',
                           cpp_args=['-E', r'-I/tmp/pycparser-master/utils/fake_libc_include', r'-I/tmp/testcasesupport/'])
ast.show()


FileAST: 
  Typedef: size_t, [], ['typedef']
    TypeDecl: size_t, []
      IdentifierType: ['int']
  Typedef: __builtin_va_list, [], ['typedef']
    TypeDecl: __builtin_va_list, []
      IdentifierType: ['int']
  Typedef: __gnuc_va_list, [], ['typedef']
    TypeDecl: __gnuc_va_list, []
      IdentifierType: ['int']
  Typedef: va_list, [], ['typedef']
    TypeDecl: va_list, []
      IdentifierType: ['int']
  Typedef: __int8_t, [], ['typedef']
    TypeDecl: __int8_t, []
      IdentifierType: ['int']
  Typedef: __uint8_t, [], ['typedef']
    TypeDecl: __uint8_t, []
      IdentifierType: ['int']
  Typedef: __int16_t, [], ['typedef']
    TypeDecl: __int16_t, []
      IdentifierType: ['int']
  Typedef: __uint16_t, [], ['typedef']
    TypeDecl: __uint16_t, []
      IdentifierType: ['int']
  Typedef: __int_least16_t, [], ['typedef']
    TypeDecl: __int_least16_t, []
      IdentifierType: ['int']
  Typedef: __uint_least16_t, [], ['typedef']
    TypeDecl: __uint_least16_t, []
      IdentifierTy

# cwe119 数据预处理

In [19]:
'''

输入：'/home/bfs/cct/astnn/data/code_cwe119_withhead'
输出：
CWE119：bad：5998 个，good：11364。
    'data/cwe119_c_ast_labeled_withfilename.pkl'
        ['id', 'name', 'code', 'ast', 'label'], (17362, 5)
    data/cwe119_c_ast_labeled_withcode.pkl'
        ['id', 'name', 'ast', 'label', 'code'], (17362, 5)
    'data/cwe119_c_ast_labeled.pkl'，ASTNN训练时所需的格式，
        ['id', 'ast', 'label']
    'data/cwe119_c_all.pkl'，包含# cwe119_asts_df.columns
        ['id', 'name', 'code', 'ast', 'label', 'corpus', 'blocks'],(17362, 7)

'data/embedding_cwe119_128.model'，CWE119 数据集的词嵌入,(458, 128)
'data/embedding_cwe399_128.model', CWE399 数据集的词嵌入,(438, 128)
'''

import pandas as pd
c = pd.read_pickle('data/cwe119_c_ast_labeled_withcode.pkl')

In [34]:
'''旧代码。不区分 bad 和 good
'''
# 遍历测试
import os
asts = {}
errors = []
# 读取所有 c/cpp 文件
i = 0
for root, dirs, files in os.walk(data_root, topdown=False):
    for name in files:
#         if name.endswith('.c') or name.endswith('.cpp'):
        if name.endswith('.c'):
            i += 1
#             if i==100: break
            try:
                ast = pycparser.parse_file(os.path.join(root, name),
                                            use_cpp=True, cpp_path='gcc',
                                            cpp_args=['-E', r'-I/tmp/pycparser-master/utils/fake_libc_include', 
                                                      r'-I/tmp/testcasesupport/'])
                asts[name] = ast
                print('%4d - successful!'%i, name)
            except Exception as e:
                errors.append(name)
                print('%4d - error! '%i, name, e)


   1 - successful! CWE126_Buffer_Overread__CWE170_char_memcpy_01.c
   2 - successful! CWE122_Heap_Based_Buffer_Overflow__c_CWE193_wchar_t_cpy_08.c
   3 - successful! CWE121_Stack_Based_Buffer_Overflow__CWE805_wchar_t_alloca_ncat_05.c
   4 - successful! CWE121_Stack_Based_Buffer_Overflow__CWE193_wchar_t_alloca_memcpy_02.c
   5 - successful! CWE121_Stack_Based_Buffer_Overflow__CWE135_03.c
   6 - successful! CWE122_Heap_Based_Buffer_Overflow__CWE131_loop_18.c
   7 - successful! CWE126_Buffer_Overread__CWE129_connect_socket_01.c
   8 - successful! CWE124_Buffer_Underwrite__wchar_t_alloca_loop_31.c
   9 - successful! CWE121_Stack_Based_Buffer_Overflow__src_wchar_t_alloca_cpy_32.c
  10 - successful! CWE415_Double_Free__malloc_free_long_32.c
  11 - successful! CWE123_Write_What_Where_Condition__fgets_05.c
  12 - successful! CWE122_Heap_Based_Buffer_Overflow__c_CWE806_wchar_t_memcpy_03.c
  13 - successful! CWE124_Buffer_Underwrite__malloc_wchar_t_ncpy_17.c
  14 - successful! CWE416_Use_After_F

In [53]:
'''
对于 cwe-119 的 
    bad： 8649 个代码文件（.c 6016，.cpp 2633），成功解析的有 5998 个，失败的有 18 个。
    good：16764（.c 11395，.cpp 5369），成功解析的有，失败的有 
    将解析生成的 .c 文件结果 FileAST 对象保存到 cwe119_c_ast.pkl
    数据结构：["source_file_name": FileAST, ... ]
    
'''
type(asts)
# 持久化到文件
pickle.dump(asts, open('data/cwe119_c_ast.pkl','wb'))

# len(errors)
# data_root 
# 保存结果
# data_root = '/home/bfs/cct/astnn/data/cwe-119_trees/'
# for k,v in asts.items():
#     with open(data_root+k, 'w') as f:
#         v.show(f)


## 区分 bad 和 good

In [9]:
"""
bad+good 共有 17411 个，成功解析 17362 个，有 49 无法解析，第二次解析，有 17362 个结果 ？

将 bad 函数和 good 函数分别作为一个单独文件进行解析。
    结果保存到 cwe119_c_ast_labeled.pkl
    数据结构：[(id, sourcefilename, code, fileast, label), ...]
仅能处理 c 代码，无法处理 cpp 代码。

Index(['id', 'name', 'code', 'ast', 'label'], dtype='object')
RangeIndex(start=0, stop=17362, step=1)
"""

import os
cwe119_pieces_withhead = '/home/bfs/cct/astnn/data/code_cwe119_withhead'

# 遍历测试
cwe119_asts = []
errors = []
# 读取所有 c/cpp 文件
i = 0
for root, dirs, files in os.walk(cwe119_pieces_withhead):
    for name in files:
        if name.endswith('.c'):
#             if i>5: break
            tmp = {}
            try:
                ast = pycparser.parse_file(os.path.join(root, name),
                                            use_cpp=True, cpp_path='gcc',
                                            cpp_args=['-E', r'-I/tmp/pycparser-master/utils/fake_libc_include', 
                                                      r'-I/tmp/testcasesupport/'])
                with open(os.path.join(root, name)) as f:
                    code = f.read()
                tmp['id'] = i
                tmp['name'] = name
                tmp['code'] = code
                tmp['ast'] = ast
                tmp['label'] = os.path.basename(root)
                cwe119_asts.append(tmp)
                i += 1
                print('%4d - successful!'%i, name)
            except Exception as e:
                errors.append(name)
                print('error! ', name, e)

   1 - successful! CWE121_Stack_Based_Buffer_Overflow__CWE131_memmove_10_goodG2B1.c
   2 - successful! CWE121_Stack_Based_Buffer_Overflow__CWE805_char_alloca_memcpy_14_goodG2B2.c
   3 - successful! CWE127_Buffer_Underread__wchar_t_declare_cpy_06_goodG2B2.c
   4 - successful! CWE122_Heap_Based_Buffer_Overflow__sizeof_struct_14_goodG2B1.c
   5 - successful! CWE122_Heap_Based_Buffer_Overflow__c_CWE129_fgets_07_goodG2B1.c
   6 - successful! CWE122_Heap_Based_Buffer_Overflow__c_CWE805_wchar_t_ncat_13_goodG2B1.c
   7 - successful! CWE122_Heap_Based_Buffer_Overflow__c_CWE806_char_memcpy_11_goodG2B1.c
   8 - successful! CWE127_Buffer_Underread__CWE839_negative_02_goodG2B2.c
   9 - successful! CWE124_Buffer_Underwrite__CWE839_connect_socket_08_goodB2G1.c
  10 - successful! CWE124_Buffer_Underwrite__wchar_t_alloca_loop_05_goodG2B2.c
  11 - successful! CWE415_Double_Free__malloc_free_int64_t_14_goodB2G2.c
  12 - successful! CWE121_Stack_Based_Buffer_Overflow__CWE805_int_declare_memcpy_08_goodG2B1

In [241]:
# 保存到文件
import pandas as pd

# cwe119_asts_df = pd.DataFrame(cwe119_asts)
cwe119_asts_df.shape
cwe119_asts_df.to_pickle('data/cwe119_c_ast_labeled_withfilename.pkl')


In [127]:
'''
Index(['id', 'name', 'code', 'ast', 'label'], dtype='object')
有些 good 函数确实一样。
为什么 code 有重复，而 ast 没有重复？？
'''# 数据检查
cwe119_asts_df = pd.DataFrame(cwe119_asts)
# # 数据唯一性检查. 17362,
# name = set(cwe119_asts_df['name'])  # 文件名是否有重复。 17362 无
# code = set(cwe119_asts_df['code'])  # 文件名是否有重复。 16702, 有
# ast = set(cwe119_asts_df['ast'])  # 文件名是否有重复。 17362, 没有，
# label = set(cwe119_asts_df['label'])  # 文件名是否有重复。 2

len(ast)

codes_dup = cwe119_asts_df.groupby(['code'])  # dict, 495 
codes_dup_size = cwe119_asts_df.groupby(['code']).size()  # pandas.core.series.Series, 16702，每个样本重复的个数
code_label = cwe119_asts_df.groupby(['code', 'label']).size()  # pandas.core.series.Series, 16702，每个样本重复的个数
# codes_dup_ = {k: tuple(d.index) for k, d in cwe119_asts_df.groupby('code') if len(d) > 1}  # 显示哪几行是一组
len(code_label)  # 16702, 很少有 函数相同且标签相同的？

16702

In [237]:
''' code 相同时，label 情况
很少有 函数相同且标签相同的？ 共有 17362个不同的 code ? ， code 和 name 都不同的 有16702个 
手动统计，结论 共有 825 个 code 是重复的，且 code 相同的 label 相同，没问题了。
'''

a1 = cwe119_asts_df['code'].values
a2 = cwe119_asts_df['label'].values
bothsame = []
singlesame = []
for i in range(len(a1)):
    for j in range(i+1, len(a1)):
        if a1[i]==a1[j] and a2[i] == a2[j]:
            bothsame.append([i,j])
        if a1[i]==a1[j] and a2[i] != a2[j]:
            singlesame.append([i,j])

print(len(bothsame))
print(len(singlesame))

825
0


In [164]:
'''
相同代码：
'\n\n\n#include "std_testcase.h"\n\n\n\nint main()\n{\n    int data;\n    \n    data = -1;\n    \n    data = 7;\n    {\n        int buffer[10] = { 0 };\n        \n        if (data < 10)\n        {\n            printIntLine(buffer[data]);\n        }\n        else\n        {\n            printLine("ERROR: Array index is too big.");\n        }\n    }\n}\n\n\n'

'''
print(cwe119_asts_df.iloc[2237, 1])
print(cwe119_asts_df.iloc[2237, 4])
cwe119_asts_df.iloc[2237, 2]

print(cwe119_asts_df.iloc[2909, 1])
print(cwe119_asts_df.iloc[2909, 4])
cwe119_asts_df.iloc[2909, 2]

print(cwe119_asts_df.iloc[4940, 1])
print(cwe119_asts_df.iloc[4940, 4])
cwe119_asts_df.iloc[4940, 2]

CWE127_Buffer_Underread__CWE839_fscanf_01_goodG2B.c
good
CWE127_Buffer_Underread__CWE839_rand_01_goodG2B.c
good
CWE127_Buffer_Underread__CWE839_negative_01_goodG2B.c
good


'\n\n\n#include "std_testcase.h"\n\n\n\nint main()\n{\n    int data;\n    \n    data = -1;\n    \n    data = 7;\n    {\n        int buffer[10] = { 0 };\n        \n        if (data < 10)\n        {\n            printIntLine(buffer[data]);\n        }\n        else\n        {\n            printLine("ERROR: Array index is too big.");\n        }\n    }\n}\n\n\n'

In [219]:
# groupby 函数测试
'''
df.groupby(列名).size() 统计某列中重复的个数，
    返回 r 是 pandas.core.series.Series 对象，
    r的长度即是该列元素种类数，第一列是值，第二列是该值的重复个数，默认按重复个数降序排序

'''
import pandas as pd
data={'state':[1,1,2,2,1,2,3],'pop':['a','b','c','d','b','c','d']}
p =pd.DataFrame(data)
print(p)
# 一列，比如查看name相同的有多少个
p_new=p.groupby(['state']).size()

# 多列，比如查看id和name都相同的有多少个
p_new=p.groupby(['state', 'pop']).size().reset_index()
p_new

   state pop
0      1   a
1      1   b
2      2   c
3      2   d
4      1   b
5      2   c
6      3   d


state pop  0
0      1   a  1
1      1   b  2
2      2   c  2
3      2   d  1
4      3   d  1

In [228]:
'''
添加 code 列。
'''
no_name = []
# cwe119_df['id'] = range(len(cwe119_df))  # 修改 id ，从 0开始
for i in range(len(cwe119_df)):
    try:
        name = cwe119_df[cwe119_df['id']==i]['name'].values[0]
        with open('data/code_cwe119_withhead/both/'+name) as f:
            code = f.read()
        cwe119_df['code'] = code
    except Exception as e:
        no_name.append(id)
        print(i, e, )
cwe119_df.to_pickle('data/cwe119_c_ast_labeled_withcode.pkl')
        

## 转成 astnn 需要的格式

In [144]:
# 转成 astnn 需要的格式
# source = pd.read_pickle('data/cwe119_c_ast_labeled.pkl')
cwe119_df_good100 = cwe119_df[cwe119_df['label']=='good']
cwe119_df_bad100 = cwe119_df[cwe119_df['label']=='bad']]
source2 = pd.concat([cwe119_df_good100, cwe119_df_bad100])

data = pd.DataFrame(source2[['id', 'ast', 'label']])
data.rename(columns={'ast':'code'}, inplace=True) 
data['id'] = range(len(data))
data['label'].replace({'good':0, 'bad':1}, inplace=True)

data.to_pickle('data/cwe119_c_ast_labeled_part-300.pkl')

17362

In [161]:
# 转成 astnn 需要的格式
def savedata(source_data, filename, good_num=-1, bad_num=-1):

    if good_num == -1:
        cwe119_df_good = cwe119_df[cwe119_df['label']=='good']
    else:
        cwe119_df_good = cwe119_df[cwe119_df['label']=='good'][:good_num]
        
    if bad_num == -1:
        cwe119_df_bad = cwe119_df[cwe119_df['label']=='bad']
    else:
        cwe119_df_bad = cwe119_df[cwe119_df['label']=='bad'][:bad_num]
    
    source = pd.concat([cwe119_df_good, cwe119_df_bad])

    data = pd.DataFrame(source[['id', 'ast', 'label']])
    data.rename(columns={'ast':'code'}, inplace=True) 
    data['id'] = range(len(data))
    data['label'].replace({'good':0, 'bad':1}, inplace=True)

    data.to_pickle(filename)
    print('save successful! ', filename)

# 仅保留部分做测试用
# savedata(cwe119_df, 'data/cwe119_c_ast_labeled_part-300.pkl', 200, 100)
# savedata(cwe119_df, 'data/cwe119_c_ast_labeled_part-3000.pkl', 2000, 1000)
savedata(cwe119_df, 'data/cwe119_c_ast_labeled.pkl')


save successful!  data/cwe119_c_ast_labeled.pkl


In [151]:
a = pd.read_pickle('data/cwe119_c_ast_labeled_part-300.pkl')
a[:3]

id                                               code  label
0   0  FileAST(ext=[Typedef(name='size_t',\n         ...      0
1   1  FileAST(ext=[Typedef(name='size_t',\n         ...      0
2   2  FileAST(ext=[Typedef(name='size_t',\n         ...      0

## 利用 astnn 生成 word2vec

In [187]:
'''
astnn 的问题：没有用到所有语料库，且没必要先划分 train、test、dev，且 3:1:1 不合理，且 128 长 不合理


'''
# 引入依赖库
astnn_vul = '/home/bfs/cct/astnn/vuldetect/'
if astnn_vul not in sys.path:
    sys.path.extend([astnn_vul])

from prepare_data import get_sequences

def trans_to_sequences(ast):
    sequence = []
    get_sequences(ast, sequence)
    return sequence

# # 生成 tokens 序列
# corpus = cwe119_asts_df['ast'].apply(trans_to_sequences)  # pandas.core.series.Series, 元素是每个 ast 中的所有 tokens ,一维列表
# str_corpus = [' '.join(c) for c in corpus]  # 二维列表，第一维长度等于样本个数，第二维是每个样本的tokens，用以空格分隔的字符串表示
# cwe119_asts_df['corpus'] = pd.Series(str_corpus)  # 将 ast 的所有 tokens 转成一个用空格分隔的字符串，如'FileAST Typedef...'

# # 计算词嵌入
# from gensim.models.word2vec import Word2Vec
# # 使用 skip-gram模型，线程数 32，词嵌入长度 128，最小词频 3
# w2v = Word2Vec(corpus, size=128, workers=32, sg=1, min_count=3)  
# w2v.save(astnn_vul+'data/embedding_128.model')

In [269]:
'''cwe119 数据集: 词嵌入结果检查
词嵌入矩阵：(458, 128),只有较少的不规范字符。和第一次只对训练集数据进行词嵌入的结果矩阵大小相同

'''
w2v.wv.syn0.shape  # (8188, 128)
# w2v.wv.syn0[:3]
# vocab = w2v.wv.vocab
# # type(vocab['fgets'])
# vocab['fgets'].index  # 该 token 的 index
# wv['fgets'] # 该 token 的embedding
# # i = [vocab[x].index for x in vocab.keys()]


/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


(458, 128)

## 生成 blocks，即带有结构信息的 ast tokens 数据

In [272]:
'''生成 blocks，即带有结构信息的 ast tokens 数据

'''
# 引入相关库
from prepare_data import get_blocks as func
from gensim.models.word2vec import Word2Vec

def tree_to_index(node):
    token = node.token
    result = [vocab[token].index if token in vocab else max_token]
    children = node.children
    for child in children:
        result.append(tree_to_index(child))
    return result

def trans2seq(r):
    blocks = []
    func(r, blocks)
    tree = []
    for b in blocks:
        btree = tree_to_index(b)
        tree.append(btree)
    return tree

# w2v = Word2Vec.load(astnn_vul+'data/embedding_128.model')
vocab = w2v.wv.vocab
max_token = w2v.wv.syn0.shape[0] # number of tokens,458

cwe119_asts_df['blocks'] = cwe119_asts_df['ast'].apply(trans2seq)


/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:27: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


In [340]:
cwe119_asts_df.to_pickle('data/cwe119_c_all.pkl')

# cwe119_asts_df.columns
# # 占用 120 MB
# sys.getsizeof(cwe119_asts_df)/1024/1024

# cwe119_asts_df['blocks']


# train

In [6]:
'''
BatchProgramClassifier
调用关系：
    model.BatchProgramClassifier，model.BatchTreeEncoder(()，都继承于：torch.nn.Module

get_batch()

'''
import sys
# 引入依赖库
astnn_vul = '/home/bfs/cct/astnn/vuldetect/'
pycparser = '/tmp/pycparser-master'
if astnn_vul not in sys.path:
    sys.path.extend([astnn_vul, pycparser])
    
import pycparser

import pandas as pd
import random
import torch
import time
import numpy as np
from gensim.models.word2vec import Word2Vec
from model import BatchProgramClassifier

from torch.autograd import Variable
from torch.utils.data import DataLoader
import os
import sys
from sklearn import metrics


# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 15
BATCH_SIZE = 256
USE_GPU = True

def get_batch(dataset, idx, bs):
    '''
    从 dataset 中 获取 1 batch 数据（ bs 条样本），从 idx 开始
    并将 labels 从 int 型转成 torch.LongTensor 型，为什么呢？ 
    返回：特征集 data，标签集 labels
    '''
    tmp = dataset.iloc[idx: idx+bs]  # 实际得到的 tmp 大小可能小于 bs
    data, labels = [], []
    for _, item in tmp.iterrows():
        data.append(item['blocks'])
        # labels.append(item[2]-1)
        labels.append(item['label'])
    return data, torch.LongTensor(labels)

# 读词嵌入
word2vec = Word2Vec.load(astnn_vul+'data/embedding_128.model').wv
embeddings = np.zeros((word2vec.syn0.shape[0] + 1, word2vec.syn0.shape[1]), dtype="float32")
embeddings[:word2vec.syn0.shape[0]] = word2vec.syn0
# embeddings 比 wv 多一行（即多一个token）,其值为全 0
MAX_TOKENS = word2vec.syn0.shape[0]  # token 种类数量
EMBEDDING_DIM = word2vec.syn0.shape[1]  # 词嵌入的维度，128维

# 创建模型，传入参数和词嵌入向量
model = BatchProgramClassifier(EMBEDDING_DIM,HIDDEN_DIM,
                                MAX_TOKENS+1,ENCODE_DIM,
                                LABELS,BATCH_SIZE,
                                USE_GPU, embeddings)
if USE_GPU:
    model.cuda()  # 转成 gpu 模型

parameters = model.parameters()  # 参数，
optimizer = torch.optim.Adamax(parameters)  # 优化器，优化模型的参数
loss_function = torch.nn.CrossEntropyLoss()  # 目标函数：交叉熵损失

best_model = model

# 读数据
cwe119_df = pd.read_pickle('data/cwe119_c_all.pkl')
cwe119_blocks_df = cwe119_df[['id', 'blocks', 'label']]
cwe119_blocks_df['label'].replace({'good':0, 'bad':1}, inplace=True)
cwe119_blocks_df_rd = cwe119_blocks_df.sample(frac=1, random_state=1)  # 随机乱序
# cwe119_blocks_df.to_pickle('data/cwe119_c_100.pkl')

# 划分比例
ratio = '7:1:2'
total_num = len(cwe119_blocks_df_rd)
ratios = [int(r) for r in ratio.split(':')]
train_split = int(ratios[0]/sum(ratios)*total_num)
val_split = train_split + int(ratios[1]/sum(ratios)*total_num)
# 划分
train_data = cwe119_blocks_df_rd.iloc[:train_split] 
val_data = cwe119_blocks_df_rd.iloc[train_split:val_split] 
test_data = cwe119_blocks_df_rd.iloc[val_split:]

/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:57: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:58: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:60: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:61: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [24]:
'''调参
训练集样本：12153，验证 1736， 测试集：3473
每轮 训练集有 12153 / 256 = 48 批数据，测试集有 3473 / 256 批数据

'''
# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 15
BATCH_SIZE = 256
USE_GPU = True

test_data.shape
3473 / 256

13.56640625

## train

In [4]:
# 训练
train_loss_ = []  # 每轮的训练损失
val_loss_ = []  # 每轮的验证损失
train_acc_ = []  # 每轮的训练acc
val_acc_ = []  # 每轮的验证acc
best_acc = 0.0  # 最高的验证acc，后面没有对它进行更新？
EPOCHS = 50
# training procedure
for epoch in range(EPOCHS):  # 每轮
    start_time = time.time()

    total_acc = total_loss = total = i = 0  # 重置标记
    # total：已训练样本的总个数，可能比 i 小。

    while i < len(train_data):  # 训练集的每批
        # 获取批数据
        batch = get_batch(train_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        train_inputs, train_labels = batch
        if USE_GPU:  # 使用 gpu 时，将 labels 转成 gpu 张量
            train_inputs, train_labels = train_inputs, train_labels.cuda()
        
        # 模型设置
        model.zero_grad()  # 把模型中参数的梯度设为0
        model.batch_size = len(train_labels)  # 为什么不用 BATCH_SIZE ？
        model.hidden = model.init_hidden()  # 后面又不用它，赋值有什么意义？
        output = model(train_inputs)  # 预测结果，二维(64, 2)，即 64 个样本，2 种标签

        loss = loss_function(output, Variable(train_labels)) # 计算损失值
        loss.backward()  # 后向传播、计算梯度，更新参数？loss 是标量。
        optimizer.step()  # 开始优化，更新参数，降低 loss，

        # calc training acc
        _, predicted = torch.max(output.data, 1)  # 返回 output 中每行（样本标签）的最大预测值和最大值的索引
        total += len(train_labels)  # 实际已训练样本个数总和
        total_acc += (predicted == train_labels).sum()  # 将所有预测正确的样本个数总和
        total_loss += loss.item()*len(train_inputs)  # 将所有样本的损失总和
        if i%(BATCH_SIZE % 20) == 0:
            # 计算 precision、recall、accuracy、f1
            y_pred = predicted.tolist()
            y_true = train_labels.tolist()
            FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
            FNR_ = 1 - recall_
            recall_ = metrics.recall_score(y_true, y_pred)
            precision_ = metrics.precision_score(y_true, y_pred)
            F1_ = metrics.f1_score(y_true, y_pred)
            print('-bad(pred/true):%4d/%4d, fpr:%.3f, fnr:%.3f, precision:%.3f, recall:%.3f, F1:%.3f' 
                  %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_))

    train_loss_.append(total_loss / total)  # 训练完一轮（所有样本）后的平均损失
    train_acc_.append(total_acc.item() / total)  # 训练完一轮（所有样本）后的平均准确率
    # 输出本轮的训练效果
#         end_time = time.time()
#     print('[Epoch: %2d/%2d] Training Loss: %.3f, Training Acc: %.3f, Time Cost: %.3f s'
#           % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], end_time - start_time))
    
    # validation epoch
    total_acc = total_loss = total = i = 0  # 重置标记
    while i < len(val_data):  # 验证集的每批，代码与训练集的处理大同小异
        batch = get_batch(val_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        val_inputs, val_labels = batch
        if USE_GPU:
            val_inputs, val_labels = val_inputs, val_labels.cuda()

        model.batch_size = len(val_labels)
        model.hidden = model.init_hidden()
        output = model(val_inputs)

        loss = loss_function(output, Variable(val_labels))

        # calc valing acc
        _, predicted = torch.max(output.data, 1)
        total_acc += (predicted == val_labels).sum()
        total += len(val_labels)
        total_loss += loss.item()*len(val_inputs)

    val_loss_.append(total_loss / total)  # 验证集训练完一轮后的平均损失
    val_acc_.append(total_acc.item() / total)  # 验证集训练完一轮后的平均acc
    times = time.time() - start_time
    if total_acc/total > best_acc:  # 没有对 best_acc 的更新，代码有问题吧
        best_model = model
        best_acc = total_acc/total
    # 输出本轮的训练、验证效果
    print('[Epoch:%2d/%2d] train_loss: %.3f, train_acc: %.3f, val_loss: %.3f, val_acc: %.3f, Time Cost: %.3f s'
          % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], val_loss_[epoch], val_acc_[epoch], times))


/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-bad(pred/true):   0/  92, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true): 113/  74, fpr:0.486, fnr:0.486, precision:0.336, recall:0.514, F1:0.406
-bad(pred/true):   0/  90, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  86, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/ 112, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  92, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  73, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  89, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/ 103, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  90, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  96, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/

## test

In [12]:
# 测试。

total_acc = total_loss = total = i = 0  # 重置标记
model = best_model  # 使用在验证集上 acc 最高的模型进行测试
while i < len(test_data):
    batch = get_batch(test_data, i, BATCH_SIZE)
    i += BATCH_SIZE
    test_inputs, test_labels = batch
    if USE_GPU:
        test_inputs, test_labels = test_inputs, test_labels.cuda()

    model.batch_size = len(test_labels)
    model.hidden = model.init_hidden()
    output = model(test_inputs)

    loss = loss_function(output, Variable(test_labels))

    _, predicted = torch.max(output.data, 1)
    total_acc += (predicted == test_labels).sum()
    total += len(test_labels)
    total_loss += loss.item() * len(test_inputs)

    # 计算 precision、recall、accuracy、f1
    y_pred = predicted.tolist()
    y_true = test_labels.tolist()
    FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
    FNR_ = 1 - recall_
    recall_ = metrics.recall_score(y_true, y_pred)
    precision_ = metrics.precision_score(y_true, y_pred)
    F1_ = metrics.f1_score(y_true, y_pred)
    print('-bad(pred/true):%4d/%4d, fpr:%.3f, fnr:%.3f, precision:%.3f, recall:%.3f, F1:%.3f' 
      %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_))

    
print("Testing results(Acc):", total_acc.item() / total)

-bad(pred/true):  94/  94, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  91/  90, fpr:0.000, fnr:0.000, precision:0.989, recall:1.000, F1:0.994
-bad(pred/true):  81/  81, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  83/  83, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  70/  70, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  78/  78, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  77/  76, fpr:0.000, fnr:0.000, precision:0.987, recall:1.000, F1:0.993
-bad(pred/true):  81/  81, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  77/  77, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  90/  90, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  91/  91, fpr:0.000, fnr:0.000, precision:1.000, recall:1.000, F1:1.000
-bad(pred/true):  97/

## train2

In [23]:
# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 30
BATCH_SIZE = 300
USE_GPU = True

# 训练
train_loss_ = []  # 每轮的训练损失
val_loss_ = []  # 每轮的验证损失
train_acc_ = []  # 每轮的训练acc
val_acc_ = []  # 每轮的验证acc
best_acc = 0.0  # 最高的验证acc，后面没有对它进行更新？
EPOCHS = 40  # 40 轮就已经达到 0.998 了
# training procedure
for epoch in range(EPOCHS):  # 每轮
    start_time = time.time()

    total_acc = total_loss = total = i = 0  # 重置标记
    # total：已训练样本的总个数，可能比 i 小。

    while i < len(train_data):  # 训练集的每批
        # 获取批数据
        batch = get_batch(train_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        train_inputs, train_labels = batch
        if USE_GPU:  # 使用 gpu 时，将 labels 转成 gpu 张量
            train_inputs, train_labels = train_inputs, train_labels.cuda()
        
        # 模型设置
        model.zero_grad()  # 把模型中参数的梯度设为0
        model.batch_size = len(train_labels)  # 为什么不用 BATCH_SIZE ？
        model.hidden = model.init_hidden()  # 后面又不用它，赋值有什么意义？
        output = model(train_inputs)  # 预测结果，二维(64, 2)，即 64 个样本，2 种标签

        loss = loss_function(output, Variable(train_labels)) # 计算损失值
        loss.backward()  # 后向传播、计算梯度，更新参数？loss 是标量。
        optimizer.step()  # 开始优化，更新参数，降低 loss，

        # calc training acc
        _, predicted = torch.max(output.data, 1)  # 返回 output 中每行（样本标签）的最大预测值和最大值的索引
        total += len(train_labels)  # 实际已训练样本个数总和
        total_acc += (predicted == train_labels).sum()  # 将所有预测正确的样本个数总和
        total_loss += loss.item()*len(train_inputs)  # 将所有样本的损失总和
        if i%(BATCH_SIZE*10) == 0:  # 每轮共有
            # 计算 precision、recall、accuracy、f1
            y_pred = predicted.tolist()
            y_true = train_labels.tolist()
            FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
            FNR_ = 1 - recall_
            recall_ = metrics.recall_score(y_true, y_pred)
            precision_ = metrics.precision_score(y_true, y_pred)
            F1_ = metrics.f1_score(y_true, y_pred)
            print('-bad(pred/true):%4d/%4d, fpr:%.3f, fnr:%.3f, precision:%.3f, recall:%.3f, F1:%.3f' 
                  %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_))

    train_loss_.append(total_loss / total)  # 训练完一轮（所有样本）后的平均损失
    train_acc_.append(total_acc.item() / total)  # 训练完一轮（所有样本）后的平均准确率
    # 输出本轮的训练效果
#         end_time = time.time()
#     print('[Epoch: %2d/%2d] Training Loss: %.3f, Training Acc: %.3f, Time Cost: %.3f s'
#           % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], end_time - start_time))
    
    # validation epoch
    total_acc = total_loss = total = i = 0  # 重置标记
    while i < len(val_data):  # 验证集的每批，代码与训练集的处理大同小异
        batch = get_batch(val_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        val_inputs, val_labels = batch
        if USE_GPU:
            val_inputs, val_labels = val_inputs, val_labels.cuda()

        model.batch_size = len(val_labels)
        model.hidden = model.init_hidden()
        output = model(val_inputs)

        loss = loss_function(output, Variable(val_labels))

        # calc valing acc
        _, predicted = torch.max(output.data, 1)
        total_acc += (predicted == val_labels).sum()
        total += len(val_labels)
        total_loss += loss.item()*len(val_inputs)

    val_loss_.append(total_loss / total)  # 验证集训练完一轮后的平均损失
    val_acc_.append(total_acc.item() / total)  # 验证集训练完一轮后的平均acc
    times = time.time() - start_time
    if total_acc/total > best_acc:  # 没有对 best_acc 的更新，代码有问题吧
        best_model = model
        best_acc = total_acc/total
    # 输出本轮的训练、验证效果
    print('[Epoch:%2d/%2d] train_loss: %.3f, train_acc: %.3f, val_loss: %.3f, val_acc: %.3f, Time Cost: %.3f s'
          % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], val_loss_[epoch], val_acc_[epoch], times))


/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-bad(pred/true):   0/ 106, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/ 112, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  91, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/  93, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
[Epoch: 1/40] train_loss: 0.638, train_acc: 0.651, val_loss: 0.613, val_acc: 0.651, Time Cost: 30.178 s
-bad(pred/true):   0/ 106, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/ 112, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   6/  91, fpr:0.956, fnr:0.956, precision:0.667, recall:0.044, F1:0.082
-bad(pred/true):   9/  93, fpr:0.957, fnr:0.957, precision:0.444, recall:0.043, F1:0.078
[Epoch: 2/40] train_loss: 0.577, train_acc: 0.652, val_loss: 0.539, val_acc: 0.697, Time Cost: 28.873 s
-bad(pred/true):  83/ 106, fpr:0.358, fnr:0.358, precision:0.819, recall:0.642, 

## test2 

In [25]:
test_data.shape

(3473, 3)

In [32]:
# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 30
BATCH_SIZE = 300
USE_GPU = True

# 测试。
total_acc = total_loss = total = i = 0  # 重置标记
model = best_model  # 使用在验证集上 acc 最高的模型进行测试
metrics_index = ['FNR_', 'FNR_', 'precision_', 'recall_', 'F1_', 'acc_']
res = {k:[] for k in ['FNR_', 'FNR_', 'precision_', 'recall_', 'F1_', 'acc_']}

while i < len(test_data):
    batch = get_batch(test_data, i, BATCH_SIZE)
    i += BATCH_SIZE
    test_inputs, test_labels = batch
    if USE_GPU:
        test_inputs, test_labels = test_inputs, test_labels.cuda()

    model.batch_size = len(test_labels)
    model.hidden = model.init_hidden()
    output = model(test_inputs)

    loss = loss_function(output, Variable(test_labels))

    _, predicted = torch.max(output.data, 1)
    total_acc += (predicted == test_labels).sum()
    total += len(test_labels)
    total_loss += loss.item() * len(test_inputs)

    # 计算 precision、recall、accuracy、f1
    y_pred = predicted.tolist()
    y_true = test_labels.tolist()
    FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
    FNR_ = 1 - recall_
    recall_ = metrics.recall_score(y_true, y_pred)
    precision_ = metrics.precision_score(y_true, y_pred)
    F1_ = metrics.f1_score(y_true, y_pred)
    acc_ = metrics.accuracy_score(y_true, y_pred)
    res['FNR_'] = FNR_
    res['FNR_'] = FNR_
    res['precision_'] = precision_
    res['recall_'] = recall_
    res['F1_'] = F1_
    res['acc_'] = acc_
    print('-bad(pred/true):%4d/%4d, fpr:%.4f, fnr:%.4f, precision:%.4f, recall:%.4f, F1:%.4f, acc:%.4f' 
      %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_, acc_))

    
print("Testing results(Acc):", total_acc.item() / total)

-bad(pred/true): 111/ 111, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  98/  97, fpr:0.0000, fnr:0.0000, precision:0.9898, recall:1.0000, F1:0.9949, acc:0.9967
-bad(pred/true): 103/ 103, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  83/  83, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  93/  93, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  87/  87, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  98/  98, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  89/  89, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true): 106/ 106, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true): 111/ 112, fpr:0.0089, fnr:0.0

## 模型持久化

In [14]:
# 保存模型
# torch.save(best_model, 'vuldetect/cwe119.model')  # 保存整个网络。报错 TypeError: can't pickle module objects
# torch.save(best_model.state_dict(), 'vuldetect/cwe119.model')  # 保存网络中的参数, 速度快，占空间少

# 加载模型

model_CKPT = torch.load('vuldetect/cwe119.model', num_worker=0)
# model.load_state_dict(model_CKPT)
#  model.eval()

# with open('vuldetect/cwe119.model') as f:
#     data = f.read()
    

TypeError: 'num_worker' is an invalid keyword argument for load()

In [14]:
''' 将数据集保存成 csv，以人工分析，为什么准确率这么高

cwe119_c_all.csv 文件大小：2.93G。太大了
Index(['id', 'name', 'code', 'ast', 'label', 'corpus', 'blocks'], dtype='object')

train_data
val_data
test_data

'''
# import pandas as pd
# cwe119_df = pd.read_pickle('data/cwe119_c_all.pkl')
# cwe119_df.to_csv('data/cwe119_c_all.csv')
cwe119_df.columns

# 划分
cwe119_part = cwe119_df[['id', 'name', 'label', 'blocks']]
cwe119_part.to_csv('data/cwe119_c_part.csv')


## 旧代码

In [166]:
# 首次将 cwe 用于 astnn
import pandas as pd
import random
import torch
import time
import numpy as np
from gensim.models.word2vec import Word2Vec
from vuldetect.model import BatchProgramClassifier
from torch.autograd import Variable
from torch.utils.data import DataLoader
import os
import sys

HIDDEN_DIM = 100
ENCODE_DIM = 128
# LABELS = 104
LABELS = 2
EPOCHS = 15
BATCH_SIZE = 64
USE_GPU = True

def get_batch(dataset, idx, bs):
    tmp = dataset.iloc[idx: idx+bs]
    data, labels = [], []
    for _, item in tmp.iterrows():
        data.append(item[1])
        # labels.append(item[2]-1)
        labels.append(item[2])
    return data, torch.LongTensor(labels)


root = 'vuldetect/data/'
train_data = pd.read_pickle(root+'train/blocks.pkl')
val_data = pd.read_pickle(root + 'dev/blocks.pkl')
test_data = pd.read_pickle(root+'test/blocks.pkl')

word2vec = Word2Vec.load(root+"train/embedding/node_w2v_128").wv
embeddings = np.zeros((word2vec.syn0.shape[0] + 1, word2vec.syn0.shape[1]), dtype="float32")
embeddings[:word2vec.syn0.shape[0]] = word2vec.syn0

MAX_TOKENS = word2vec.syn0.shape[0]
EMBEDDING_DIM = word2vec.syn0.shape[1]

model = BatchProgramClassifier(EMBEDDING_DIM,HIDDEN_DIM,MAX_TOKENS+1,ENCODE_DIM,LABELS,BATCH_SIZE,
                               USE_GPU, embeddings)
if USE_GPU:
    model.cuda()

parameters = model.parameters()
optimizer = torch.optim.Adamax(parameters)
loss_function = torch.nn.CrossEntropyLoss()

train_loss_ = []
val_loss_ = []
train_acc_ = []
val_acc_ = []
best_acc = 0.0
print('Start training...')
# training procedure
best_model = model
for epoch in range(EPOCHS):
    start_time = time.time()

    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    i = 0
    while i < len(train_data):
        batch = get_batch(train_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        train_inputs, train_labels = batch
        if USE_GPU:
            train_inputs, train_labels = train_inputs, train_labels.cuda()

        model.zero_grad()
        model.batch_size = len(train_labels)
        model.hidden = model.init_hidden()
        output = model(train_inputs)

        loss = loss_function(output, Variable(train_labels))
        loss.backward()
        optimizer.step()

        # calc training acc
        _, predicted = torch.max(output.data, 1)
        total_acc += (predicted == train_labels).sum()
        total += len(train_labels)
        total_loss += loss.item()*len(train_inputs)

    train_loss_.append(total_loss / total)
    train_acc_.append(total_acc.item() / total)
    # validation epoch
    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    i = 0
    while i < len(val_data):
        batch = get_batch(val_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        val_inputs, val_labels = batch
        if USE_GPU:
            val_inputs, val_labels = val_inputs, val_labels.cuda()

        model.batch_size = len(val_labels)
        model.hidden = model.init_hidden()
        output = model(val_inputs)

        loss = loss_function(output, Variable(val_labels))

        # calc valing acc
        _, predicted = torch.max(output.data, 1)
        total_acc += (predicted == val_labels).sum()
        total += len(val_labels)
        total_loss += loss.item()*len(val_inputs)
    val_loss_.append(total_loss / total)
    val_acc_.append(total_acc.item() / total)
    end_time = time.time()
    if total_acc/total > best_acc:
        best_model = model
    print('[Epoch: %3d/%3d] Training Loss: %.4f, Validation Loss: %.4f,'
          ' Training Acc: %.3f, Validation Acc: %.3f, Time Cost: %.3f s'
          % (epoch + 1, EPOCHS, train_loss_[epoch], val_loss_[epoch],
             train_acc_[epoch], val_acc_[epoch], end_time - start_time))

total_acc = 0.0
total_loss = 0.0
total = 0.0
i = 0
model = best_model
while i < len(test_data):
    batch = get_batch(test_data, i, BATCH_SIZE)
    i += BATCH_SIZE
    test_inputs, test_labels = batch
    if USE_GPU: 
        test_inputs, test_labels = test_inputs, test_labels.cuda()

    model.batch_size = len(test_labels)
    model.hidden = model.init_hidden()
    output = model(test_inputs)

    loss = loss_function(output, Variable(test_labels))

    _, predicted = torch.max(output.data, 1)
    total_acc += (predicted == test_labels).sum()
    total += len(test_labels)
    total_loss += loss.item() * len(test_inputs)
print("Testing results(Acc):", total_acc.item() / total)


/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:37: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:38: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:40: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/ipykernel/__main__.py:41: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


Start training...
[Epoch:   1/ 15] Training Loss: 0.5233, Validation Loss: 0.2873, Training Acc: 0.734, Validation Acc: 0.883, Time Cost: 70.766 s
[Epoch:   2/ 15] Training Loss: 0.2047, Validation Loss: 0.1407, Training Acc: 0.903, Validation Acc: 0.948, Time Cost: 71.677 s
[Epoch:   3/ 15] Training Loss: 0.1133, Validation Loss: 0.0843, Training Acc: 0.951, Validation Acc: 0.958, Time Cost: 66.207 s
[Epoch:   4/ 15] Training Loss: 0.0751, Validation Loss: 0.0800, Training Acc: 0.968, Validation Acc: 0.961, Time Cost: 72.571 s
[Epoch:   5/ 15] Training Loss: 0.0615, Validation Loss: 0.0600, Training Acc: 0.972, Validation Acc: 0.971, Time Cost: 73.164 s
[Epoch:   6/ 15] Training Loss: 0.0512, Validation Loss: 0.0518, Training Acc: 0.977, Validation Acc: 0.973, Time Cost: 71.473 s
[Epoch:   7/ 15] Training Loss: 0.0404, Validation Loss: 0.0377, Training Acc: 0.984, Validation Acc: 0.981, Time Cost: 66.630 s
[Epoch:   8/ 15] Training Loss: 0.0320, Validation Loss: 0.0272, Training Acc: 

In [ ]:
''' 调错记录
用于漏洞检测时报错：RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  
报错行： loss = loss_function(output, Variable(train_labels))
分析：
    源任务是 104 类标签，
    将 train.py 中的 LABELS = 104 改为 2 后，仍然报错

网友的解释：
    直观上看，函数要求目标分类数大于等于0并且小于等于输入的类别。
    所以一般而言，都是网络中输出的种类数和标签中设置的种类数量不同造成的。
    方向一：模型输出与分类数不一致
        看一下模型的输出尺寸与分类数差异是否明显，核查代码是否存在错误。
        模型的输出尺寸：output.ndim 为 2，train_labels 的结果是 0 和 -1。是否要改成 0 和 1 ？ 如何修改？
        溯源，找到了 labels.append(item[2]-1)，修改为 labels.append(item[2])。解决
        如果没有错误，只是映射维度不对，可以考虑在模型的最后一层加一层FC层，将输出尺寸映射到分类大小。
    方向二：标签的设置不是从0开始
        如果模型的输出尺寸与分类数大小相同，看一下标签的设定是否是从0开始的。
        如果标签是从1开始设置的，重新设置标签。这里存在的坑是：在使用CrossEntropyLoss()这个函数进行验证时，标签必须从0开始设置，否则便会报错。

    一般来说在我们网络中输出的种类数和你label设置的种类数量不同的时候就会出现这个错误。
    但是，Pytorch有个要求，在使用CrossEntropyLoss这个函数进行验证时label必须是以0开始的：
    假如我这样:
        self.classes = [0, 1, 2, 3]
    我的种类有四类，分别是0.1.2.3，这样就没有什么问题，但是如果我写成：
        self.classes = [1, 2, 3, 4]
    这样就会报错


'''

## cwe119 4:1:5


In [1]:
'''
BatchProgramClassifier
调用关系：
    model.BatchProgramClassifier，model.BatchTreeEncoder(()，都继承于：torch.nn.Module

get_batch()

'''
import sys
# 引入依赖库
astnn_vul = '/home/bfs/cct/astnn/vuldetect/'
pycparser = '/tmp/pycparser-master'
if astnn_vul not in sys.path:
    sys.path.extend([astnn_vul, pycparser])
    
import pycparser

import pandas as pd
import random
import torch
import time
import numpy as np
from gensim.models.word2vec import Word2Vec
from model import BatchProgramClassifier

from torch.autograd import Variable
from torch.utils.data import DataLoader
import os
import sys
from sklearn import metrics


# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 15
BATCH_SIZE = 256
USE_GPU = True

def get_batch(dataset, idx, bs):
    '''
    从 dataset 中 获取 1 batch 数据（ bs 条样本），从 idx 开始
    并将 labels 从 int 型转成 torch.LongTensor 型，为什么呢？ 
    返回：特征集 data，标签集 labels
    '''
    tmp = dataset.iloc[idx: idx+bs]  # 实际得到的 tmp 大小可能小于 bs
    data, labels = [], []
    for _, item in tmp.iterrows():
        data.append(item['blocks'])
        # labels.append(item[2]-1)
        labels.append(item['label'])
    return data, torch.LongTensor(labels)

# 读词嵌入
word2vec = Word2Vec.load('data/embedding_128.model').wv
embeddings = np.zeros((word2vec.vectors.shape[0] + 1, word2vec.vectors.shape[1]), dtype="float32")
embeddings[:word2vec.vectors.shape[0]] = word2vec.vectors
# embeddings 比 wv 多一行（即多一个token）,其值为全 0
MAX_TOKENS = word2vec.vectors.shape[0]  # token 种类数量
EMBEDDING_DIM = word2vec.vectors.shape[1]  # 词嵌入的维度，128维

# 创建模型，传入参数和词嵌入向量
model = BatchProgramClassifier(EMBEDDING_DIM,HIDDEN_DIM,
                                MAX_TOKENS+1,ENCODE_DIM,
                                LABELS,BATCH_SIZE,
                                USE_GPU, embeddings)
if USE_GPU:
    model.cuda()  # 转成 gpu 模型

parameters = model.parameters()  # 参数，
optimizer = torch.optim.Adamax(parameters)  # 优化器，优化模型的参数
loss_function = torch.nn.CrossEntropyLoss()  # 目标函数：交叉熵损失

best_model = model

# 读数据
cwe119_df = pd.read_pickle('data/cwe119_c_all.pkl')
cwe119_blocks_df = cwe119_df[['id', 'blocks', 'label']]
cwe119_blocks_df['label'].replace({'good':0, 'bad':1}, inplace=True)
cwe119_blocks_df_rd = cwe119_blocks_df.sample(frac=1, random_state=1)  # 随机乱序
# cwe119_blocks_df.to_pickle('data/cwe119_c_100.pkl')

# 划分比例
ratio = '4:1:5'
total_num = len(cwe119_blocks_df_rd)
ratios = [int(r) for r in ratio.split(':')]
train_split = int(ratios[0]/sum(ratios)*total_num)
val_split = train_split + int(ratios[1]/sum(ratios)*total_num)
# 划分
train_data = cwe119_blocks_df_rd.iloc[:train_split] 
val_data = cwe119_blocks_df_rd.iloc[train_split:val_split] 
test_data = cwe119_blocks_df_rd.iloc[val_split:]

# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 30
BATCH_SIZE = 300
USE_GPU = True

# 训练
train_loss_ = []  # 每轮的训练损失
val_loss_ = []  # 每轮的验证损失
train_acc_ = []  # 每轮的训练acc
val_acc_ = []  # 每轮的验证acc
best_acc = 0.0  # 最高的验证acc，后面没有对它进行更新？
EPOCHS = 40  # 40 轮就已经达到 0.998 了
# training procedure
for epoch in range(EPOCHS):  # 每轮
    start_time = time.time()

    total_acc = total_loss = total = i = 0  # 重置标记
    # total：已训练样本的总个数，可能比 i 小。

    while i < len(train_data):  # 训练集的每批
        # 获取批数据
        batch = get_batch(train_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        train_inputs, train_labels = batch
        if USE_GPU:  # 使用 gpu 时，将 labels 转成 gpu 张量
            train_inputs, train_labels = train_inputs, train_labels.cuda()
        
        # 模型设置
        model.zero_grad()  # 把模型中参数的梯度设为0
        model.batch_size = len(train_labels)  # 为什么不用 BATCH_SIZE ？
        model.hidden = model.init_hidden()  # 后面又不用它，赋值有什么意义？
        output = model(train_inputs)  # 预测结果，二维(64, 2)，即 64 个样本，2 种标签

        loss = loss_function(output, Variable(train_labels)) # 计算损失值
        loss.backward()  # 后向传播、计算梯度，更新参数？loss 是标量。
        optimizer.step()  # 开始优化，更新参数，降低 loss，

        # calc training acc
        _, predicted = torch.max(output.data, 1)  # 返回 output 中每行（样本标签）的最大预测值和最大值的索引
        total += len(train_labels)  # 实际已训练样本个数总和
        total_acc += (predicted == train_labels).sum()  # 将所有预测正确的样本个数总和
        total_loss += loss.item()*len(train_inputs)  # 将所有样本的损失总和
        if i%(BATCH_SIZE*10) == 0:  # 每轮共有
            # 计算 precision、recall、accuracy、f1
            y_pred = predicted.tolist()
            y_true = train_labels.tolist()
            FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
            FNR_ = 1 - recall_
            recall_ = metrics.recall_score(y_true, y_pred)
            precision_ = metrics.precision_score(y_true, y_pred)
            F1_ = metrics.f1_score(y_true, y_pred)
            print('-bad(pred/true):%4d/%4d, fpr:%.3f, fnr:%.3f, precision:%.3f, recall:%.3f, F1:%.3f' 
                  %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_))

    train_loss_.append(total_loss / total)  # 训练完一轮（所有样本）后的平均损失
    train_acc_.append(total_acc.item() / total)  # 训练完一轮（所有样本）后的平均准确率
    # 输出本轮的训练效果
#         end_time = time.time()
#     print('[Epoch: %2d/%2d] Training Loss: %.3f, Training Acc: %.3f, Time Cost: %.3f s'
#           % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], end_time - start_time))
    
    # validation epoch
    total_acc = total_loss = total = i = 0  # 重置标记
    while i < len(val_data):  # 验证集的每批，代码与训练集的处理大同小异
        batch = get_batch(val_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        val_inputs, val_labels = batch
        if USE_GPU:
            val_inputs, val_labels = val_inputs, val_labels.cuda()

        model.batch_size = len(val_labels)
        model.hidden = model.init_hidden()
        output = model(val_inputs)

        loss = loss_function(output, Variable(val_labels))

        # calc valing acc
        _, predicted = torch.max(output.data, 1)
        total_acc += (predicted == val_labels).sum()
        total += len(val_labels)
        total_loss += loss.item()*len(val_inputs)

    val_loss_.append(total_loss / total)  # 验证集训练完一轮后的平均损失
    val_acc_.append(total_acc.item() / total)  # 验证集训练完一轮后的平均acc
    times = time.time() - start_time
    if total_acc/total > best_acc:  # 没有对 best_acc 的更新，代码有问题吧
        best_model = model
        best_acc = total_acc/total
    # 输出本轮的训练、验证效果
    print('[Epoch:%2d/%2d] train_loss: %.3f, train_acc: %.3f, val_loss: %.3f, val_acc: %.3f, Time Cost: %.3f s'
          % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], val_loss_[epoch], val_acc_[epoch], times))


/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/bfs/anaconda3/envs/astnn2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-bad(pred/true):   0/ 106, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/ 112, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
[Epoch: 1/40] train_loss: 0.662, train_acc: 0.633, val_loss: 0.641, val_acc: 0.643, Time Cost: 18.954 s
-bad(pred/true):   0/ 106, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   0/ 112, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
[Epoch: 2/40] train_loss: 0.626, train_acc: 0.647, val_loss: 0.614, val_acc: 0.643, Time Cost: 17.346 s
-bad(pred/true):   0/ 106, fpr:0.000, fnr:0.000, precision:0.000, recall:0.000, F1:0.000
-bad(pred/true):   3/ 112, fpr:0.973, fnr:0.973, precision:1.000, recall:0.027, F1:0.052
[Epoch: 3/40] train_loss: 0.578, train_acc: 0.655, val_loss: 0.542, val_acc: 0.759, Time Cost: 16.980 s
-bad(pred/true):  68/ 106, fpr:0.443, fnr:0.443, precision:0.868, recall:0.557, F1:0.678
-bad(pred/true):  93/ 112, fpr:0.357, fnr:0.357, precision:0.774,

In [2]:
# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 30
BATCH_SIZE = 300
USE_GPU = True

# 测试。
total_acc = total_loss = total = i = 0  # 重置标记
model = best_model  # 使用在验证集上 acc 最高的模型进行测试
metrics_index = ['FNR_', 'FNR_', 'precision_', 'recall_', 'F1_', 'acc_']
res = {k:[] for k in ['FNR_', 'FNR_', 'precision_', 'recall_', 'F1_', 'acc_']}

while i < len(test_data):
    batch = get_batch(test_data, i, BATCH_SIZE)
    i += BATCH_SIZE
    test_inputs, test_labels = batch
    if USE_GPU:
        test_inputs, test_labels = test_inputs, test_labels.cuda()

    model.batch_size = len(test_labels)
    model.hidden = model.init_hidden()
    output = model(test_inputs)

    loss = loss_function(output, Variable(test_labels))

    _, predicted = torch.max(output.data, 1)
    total_acc += (predicted == test_labels).sum()
    total += len(test_labels)
    total_loss += loss.item() * len(test_inputs)

    # 计算 precision、recall、accuracy、f1
    y_pred = predicted.tolist()
    y_true = test_labels.tolist()
    FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
    FNR_ = 1 - recall_
    recall_ = metrics.recall_score(y_true, y_pred)
    precision_ = metrics.precision_score(y_true, y_pred)
    F1_ = metrics.f1_score(y_true, y_pred)
    acc_ = metrics.accuracy_score(y_true, y_pred)
    res['FNR_'] = FNR_
    res['FNR_'] = FNR_
    res['precision_'] = precision_
    res['recall_'] = recall_
    res['F1_'] = F1_
    res['acc_'] = acc_
    print('-bad(pred/true):%4d/%4d, fpr:%.4f, fnr:%.4f, precision:%.4f, recall:%.4f, F1:%.4f, acc:%.4f' 
      %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_, acc_))

    
print("Testing results(Acc):", total_acc.item() / total)

-bad(pred/true):  96/  96, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  91/  91, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true): 115/ 114, fpr:0.0000, fnr:0.0000, precision:0.9913, recall:1.0000, F1:0.9956, acc:0.9967
-bad(pred/true):  93/  92, fpr:0.0000, fnr:0.0000, precision:0.9892, recall:1.0000, F1:0.9946, acc:0.9967
-bad(pred/true):  96/  96, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  95/  92, fpr:0.0000, fnr:0.0000, precision:0.9684, recall:1.0000, F1:0.9840, acc:0.9900
-bad(pred/true): 109/ 107, fpr:0.0000, fnr:0.0000, precision:0.9817, recall:1.0000, F1:0.9907, acc:0.9933
-bad(pred/true): 111/ 110, fpr:0.0000, fnr:0.0000, precision:0.9910, recall:1.0000, F1:0.9955, acc:0.9967
-bad(pred/true): 119/ 117, fpr:0.0000, fnr:0.0000, precision:0.9832, recall:1.0000, F1:0.9915, acc:0.9933
-bad(pred/true): 108/ 107, fpr:0.0000, fnr:0.0

# cwe399

## 数据预处理

In [1]:
""" 1.1 生成 AST。仅能处理 c 代码，无法处理 cpp 代码。

输入：从cwe399 数据集（4560个）提取/拆分出：bad：1408，good：3321 （共计 4729）的样本
处理：
    bad+good 共有 4729 个，
    第一次：成功解析 1399 个，有 38 无法解析。
    第二次：成功解析 4570，error count:  159
输出
    结果保存到 cwe119_c_ast_labeled.pkl
    (4570, 5)
    Index(['id', 'name', 'code', 'ast', 'label'], dtype='object')

"""
import sys
pycparser_path = '/tmp/pycparser-master'
if pycparser_path not in sys.path:
    sys.path.extend([pycparser_path])
import pycparser

import os
cwe399_pieces_withhead = '/home/bfs/cct/astnn/data/code_cwe399_withhead'

# 先将文件名保存到列表，再按列表的顺序处理。方便断点续做
cwe399_c_filenames = []
for root, dirs, files in os.walk(cwe399_pieces_withhead):
    for file in files:
        if file.endswith('.c'):
            cwe399_c_filenames.append(os.path.join(root, file))

i = c = 0
cwe399_asts = []
cwe399_errors = []
for name in cwe399_c_filenames:
    c += 1
#     if c == 10: break
    try:
        ast = pycparser.parse_file(name, use_cpp=True, cpp_path='gcc',
                                    cpp_args=['-E', r'-I/tmp/pycparser-master/utils/fake_libc_include', 
                                              r'-I/tmp/testcasesupport/'])
        with open(name) as f:
            code = f.read()
        tmp = {}
        tmp['id'] = i
        tmp['name'] = name
        tmp['code'] = code
        tmp['ast'] = ast
        tmp['label'] = os.path.basename(os.path.dirname(name))
        cwe399_asts.append(tmp)
        i += 1
        print('%4d - successful!'%i, name)
    except Exception as e:
        cwe399_errors.append(name)
        print('error! ', e, name)

   1 - successful! /home/bfs/cct/astnn/data/code_cwe399_withhead/good/CWE789_Uncontrolled_Mem_Alloc__malloc_char_fscanf_10_goodB2G2.c
error!  Command '['gcc', '-E', '-I/tmp/pycparser-master/utils/fake_libc_include', '-I/tmp/testcasesupport/', '/home/bfs/cct/astnn/data/code_cwe399_withhead/good/CWE775_Missing_Release_of_File_Descriptor_or_Handle__w32CreateFile_no_close_18_goodB2G.c']' returned non-zero exit status 1. /home/bfs/cct/astnn/data/code_cwe399_withhead/good/CWE775_Missing_Release_of_File_Descriptor_or_Handle__w32CreateFile_no_close_18_goodB2G.c
   2 - successful! /home/bfs/cct/astnn/data/code_cwe399_withhead/good/CWE590_Free_Memory_Not_on_Heap__free_int64_t_static_17_goodG2B.c
   3 - successful! /home/bfs/cct/astnn/data/code_cwe399_withhead/good/CWE775_Missing_Release_of_File_Descriptor_or_Handle__open_no_close_16_goodB2G.c
   4 - successful! /home/bfs/cct/astnn/data/code_cwe399_withhead/good/CWE401_Memory_Leak__int64_t_calloc_11_goodB2G2.c
error!  Command '['gcc', '-E', '-I/t

In [13]:
''' ast 结果分析与保存
cwe399_asts = []
cwe399_errors = []
'''
    
c = 0
for root, dirs, files in os.walk(cwe399_pieces_withhead):
    for name in files:
        if name.endswith('.c'):
#         or name.endswith('.cpp'):
            c += 1

print('source count:', c)            
print('successful count: ', len(cwe399_asts))
print('error count: ', len(cwe399_errors))

# # 保存到文件
import pandas as pd
cwe399_asts_df = pd.DataFrame(cwe399_asts)
cwe399_asts_df.shape
cwe399_asts_df.columns
cwe399_asts_df.to_pickle('data/cwe399_c_ast_labeled_withfilename.pkl')

# # cwe399_asts_df = pd.read_pickle('data/cwe399_c_ast_labeled_withfilename.pkl')
# cwe399_asts_df.iloc[1,1]
# cwe399_asts_df.iloc[5,1]


source count: 4729
successful count:  4570
error count:  159


In [ ]:
""" 1.1 生成 AST（多进程）
单进程处理 4570 耗时 54m 26s。
服务器是 双cpu、各14核心，共 56线程
"""
def get_ast(filename):
    tmp = {}
    try:
        ast = pycparser.parse_file(filename, use_cpp=True, cpp_path='gcc',
                                   cpp_args=['-E', r'-I/tmp/pycparser-master/utils/fake_libc_include', 
                                             r'-I/tmp/testcasesupport/'])
        with open(filename) as f:
            code = f.read()
#         tmp['id'] = i
        tmp['name'] = filename
        tmp['code'] = code
        tmp['ast'] = ast
        tmp['label'] = os.path.basename(os.path.dirname(filename))
        print('%4d - successful!'%i, name)
    except Exception as e:
        cwe399_errors.append(name)
        print('error! ', e, name)
        return 'error! ' + e
    return tmp

# 接受函数的返回值
cwe399_asts = []
cwe399_errors = []
# 创建进程池，分配任务
pool = Pool(processes = 8 )    # 维护执行的进程4个
for name in cwe399_c_filenames:
    cwe399_asts.append(pool.apply_async(get_ast, (name,)))  #分配进程任务，非阻塞
    # pool.apply(fun, (i,) )  #分配进程任务，阻塞
print("Allocation of multi-process-tasks completed.")
pool.close()    # 执行完close后不会有新的进程加入到pool,
pool.join()     # 调用join之前，先调用close函数，否则会出错。join函数等待所有子进程结束


In [14]:
''' 生成 词嵌入
astnn 的问题：没有用到所有语料库，且没必要先划分 train、test、dev，且 3:1:1 不合理，且 128 长 不合理


'''
# 引入依赖库
astnn_vul = '/home/bfs/cct/astnn/vuldetect/'
if astnn_vul not in sys.path:
    sys.path.extend([astnn_vul])

from prepare_data import get_sequences

def trans_to_sequences(ast):
    sequence = []
    get_sequences(ast, sequence)
    return sequence

# 生成 tokens 序列
cwe399_corpus = cwe399_asts_df['ast'].apply(trans_to_sequences)  # 元素是每个 ast 中的所有 tokens，一维列表
# 将 tokens 列表 转成 tokens 字符串
cwe399_asts_df['corpus'] = pd.Series([' '.join(c) for c in cwe399_corpus])

# 计算词嵌入
from gensim.models.word2vec import Word2Vec
# 使用 skip-gram模型，线程数 32，词嵌入长度 128，最小词频 3
w2v = Word2Vec(cwe399_corpus, size=128, workers=32, sg=1, min_count=3)  



In [34]:
'''词嵌入结果检查（ cwe399 数据集）:
词嵌入矩阵：(292, 128),只有较少的不规范字符。和第一次只对训练集数据进行词嵌入的结果矩阵大小相同
第二次：(438, 128)

'''
w2v.wv.vectors.shape  # (292, 128)

# # # 生成 词嵌入 mapping: { index : token }
# mapping = {w2v.wv.vocab[x].index:x for x in w2v.wv.vocab.keys()}
# with open('data/mapping_cwe399.txt', 'w') as f:
#     f.write(str(mapping))
# # cwe399_asts_df['mapping'] = mapping
# # # 保存
# w2v.save('data/embedding_cwe399_128.model')



(438, 128)

In [31]:
'''生成 blocks，即带有结构信息的 ast tokens 数据

'''
# 引入相关库
from prepare_data import get_blocks as func
from gensim.models.word2vec import Word2Vec

def tree_to_index(node):
    token = node.token
    result = [vocab[token].index if token in vocab else max_token]
    children = node.children
    for child in children:
        result.append(tree_to_index(child))
    return result

def trans2seq(r):
    blocks = []
    func(r, blocks)
    tree = []
    for b in blocks:
        btree = tree_to_index(b)
        tree.append(btree)
    return tree

# w2v = Word2Vec.load(astnn_vul+'data/embedding_cwe399_128.model')
vocab = w2v.wv.vocab
max_token = w2v.wv.vectors.shape[0] # number of tokens,458

cwe399_asts_df['blocks'] = cwe399_asts_df['ast'].apply(trans2seq)


In [38]:
''' 数据预处理结果分析与保存（cwe399）
(4570, 7)
Index(['id', 'name', 'code', 'ast', 'label', 'corpus', 'blocks'], dtype='object')

'''
 
# # 占用 33. MB
sys.getsizeof(cwe399_asts_df)/1024/1024  # 单位MB
# cwe399_asts_df.columns
# cwe399_asts_df.shape
cwe399_asts_df.to_pickle('data/cwe399_c_all.pkl')
# cwe399_asts_df.to_csv('data/cwe399_c_all.csv')


## train

In [47]:
'''
训练 30 轮 就已经达到 0.998 acc 了。

'''
import sys
# 引入依赖库
astnn_vul = '/home/bfs/cct/astnn/vuldetect/'
pycparser = '/tmp/pycparser-master'
if astnn_vul not in sys.path:
    sys.path.extend([astnn_vul, pycparser])
    
import pycparser

import pandas as pd
import random
import torch
import time
import numpy as np
from gensim.models.word2vec import Word2Vec
from model import BatchProgramClassifier

from torch.autograd import Variable
from torch.utils.data import DataLoader
import os
import sys
from sklearn import metrics

# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 15
BATCH_SIZE = 256
USE_GPU = True

def get_batch(dataset, idx, bs):
    '''
    从 dataset 中 获取 1 batch 数据（ bs 条样本），从 idx 开始
    并将 labels 从 int 型转成 torch.LongTensor 型，为什么呢？ 
    返回：特征集 data，标签集 labels
    '''
    tmp = dataset.iloc[idx: idx+bs]  # 实际得到的 tmp 大小可能小于 bs
    data, labels = [], []
    for _, item in tmp.iterrows():
        data.append(item['blocks'])
        # labels.append(item[2]-1)
        labels.append(item['label'])
    return data, torch.LongTensor(labels)

# 读词嵌入
word2vec = Word2Vec.load('data/embedding_cwe399_128.model').wv
embeddings = np.zeros((word2vec.vectors.shape[0] + 1, word2vec.vectors.shape[1]), dtype="float32")
embeddings[:word2vec.vectors.shape[0]] = word2vec.vectors
# embeddings 比 wv 多一行（即多一个token）,其值为全 0
MAX_TOKENS = word2vec.vectors.shape[0]  # token 种类数量
EMBEDDING_DIM = word2vec.vectors.shape[1]  # 词嵌入的维度，128维

# 创建模型，传入参数和词嵌入向量
model = BatchProgramClassifier(EMBEDDING_DIM,HIDDEN_DIM,
                                MAX_TOKENS+1,ENCODE_DIM,
                                LABELS,BATCH_SIZE,
                                USE_GPU, embeddings)
if USE_GPU:
    model.cuda()  # 转成 gpu 模型

parameters = model.parameters()  # 参数，
optimizer = torch.optim.Adamax(parameters)  # 优化器，优化模型的参数
loss_function = torch.nn.CrossEntropyLoss()  # 目标函数：交叉熵损失

best_model = model

# # 读数据
# cwe399_df = pd.read_pickle('data/cwe399_c_all.pkl')
# cwe399_blocks_df = cwe399_df[['id', 'blocks', 'label']]
# cwe399_blocks_df['label'].replace({'good':0, 'bad':1}, inplace=True)
# cwe399_blocks_df_rd = cwe399_blocks_df.sample(frac=1, random_state=1)  # 随机乱序
# # cwe119_blocks_df.to_pickle('data/cwe119_c_100.pkl')

# # 划分比例
# ratio = '7:1:2'
# total_num = len(cwe399_blocks_df_rd)
# ratios = [int(r) for r in ratio.split(':')]
# train_split = int(ratios[0]/sum(ratios)*total_num)
# val_split = train_split + int(ratios[1]/sum(ratios)*total_num)
# # 划分
# train_data = cwe399_blocks_df_rd.iloc[:train_split] 
# val_data = cwe399_blocks_df_rd.iloc[train_split:val_split] 
# test_data = cwe399_blocks_df_rd.iloc[val_split:]

In [45]:
train_data.shapeo

(3199, 3)

In [50]:
# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 20
BATCH_SIZE = 300
USE_GPU = True

# 训练
train_loss_ = []  # 每轮的训练损失
val_loss_ = []  # 每轮的验证损失
train_acc_ = []  # 每轮的训练acc
val_acc_ = []  # 每轮的验证acc
best_acc = 0.0  # 最高的验证acc，后面没有对它进行更新？
EPOCHS = 40  # 40 轮就已经达到 0.998 了
# training procedure
for epoch in range(EPOCHS):  # 每轮
    start_time = time.time()

    total_acc = total_loss = total = i = 0  # 重置标记
    # total：已训练样本的总个数，可能比 i 小。

    while i < len(train_data):  # 训练集的每批
        # 获取批数据
        batch = get_batch(train_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        train_inputs, train_labels = batch
        if USE_GPU:  # 使用 gpu 时，将 labels 转成 gpu 张量
            train_inputs, train_labels = train_inputs, train_labels.cuda()
        
        # 模型设置
        model.zero_grad()  # 把模型中参数的梯度设为0
        model.batch_size = len(train_labels)  # 为什么不用 BATCH_SIZE ？
        model.hidden = model.init_hidden()  # 后面又不用它，赋值有什么意义？
        output = model(train_inputs)  # 预测结果，二维(64, 2)，即 64 个样本，2 种标签

        loss = loss_function(output, Variable(train_labels)) # 计算损失值
        loss.backward()  # 后向传播、计算梯度，更新参数？loss 是标量。
        optimizer.step()  # 开始优化，更新参数，降低 loss，

        # calc training acc
        _, predicted = torch.max(output.data, 1)  # 返回 output 中每行（样本标签）的最大预测值和最大值的索引
        total += len(train_labels)  # 实际已训练样本个数总和
        total_acc += (predicted == train_labels).sum()  # 将所有预测正确的样本个数总和
        total_loss += loss.item()*len(train_inputs)  # 将所有样本的损失总和
        if i%(BATCH_SIZE*5) == 0:  # 每轮共有
            # 计算 precision、recall、accuracy、f1
            y_pred = predicted.tolist()
            y_true = train_labels.tolist()
            FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
            FNR_ = 1 - recall_
            recall_ = metrics.recall_score(y_true, y_pred)
            precision_ = metrics.precision_score(y_true, y_pred)
            F1_ = metrics.f1_score(y_true, y_pred)
            print('-bad(pred/true):%4d/%4d, fpr:%.3f, fnr:%.3f, precision:%.3f, recall:%.3f, F1:%.3f' 
                  %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_))

    train_loss_.append(total_loss / total)  # 训练完一轮（所有样本）后的平均损失
    train_acc_.append(total_acc.item() / total)  # 训练完一轮（所有样本）后的平均准确率
    # 输出本轮的训练效果
#         end_time = time.time()
#     print('[Epoch: %2d/%2d] Training Loss: %.3f, Training Acc: %.3f, Time Cost: %.3f s'
#           % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], end_time - start_time))
    
    # validation epoch
    total_acc = total_loss = total = i = 0  # 重置标记
    while i < len(val_data):  # 验证集的每批，代码与训练集的处理大同小异
        batch = get_batch(val_data, i, BATCH_SIZE)
        i += BATCH_SIZE
        val_inputs, val_labels = batch
        if USE_GPU:
            val_inputs, val_labels = val_inputs, val_labels.cuda()

        model.batch_size = len(val_labels)
        model.hidden = model.init_hidden()
        output = model(val_inputs)

        loss = loss_function(output, Variable(val_labels))

        # calc valing acc
        _, predicted = torch.max(output.data, 1)
        total_acc += (predicted == val_labels).sum()
        total += len(val_labels)
        total_loss += loss.item()*len(val_inputs)

    val_loss_.append(total_loss / total)  # 验证集训练完一轮后的平均损失
    val_acc_.append(total_acc.item() / total)  # 验证集训练完一轮后的平均acc
    times = time.time() - start_time
    if total_acc/total > best_acc:  # 没有对 best_acc 的更新，代码有问题吧
        best_model = model
        best_acc = total_acc/total
    # 输出本轮的训练、验证效果
    print('[Epoch:%2d/%2d] train_loss: %.3f, train_acc: %.3f, val_loss: %.3f, val_acc: %.3f, Time Cost: %.3f s'
          % (epoch + 1, EPOCHS, train_loss_[epoch], train_acc_[epoch], val_loss_[epoch], val_acc_[epoch], times))


-bad(pred/true):  51/  95, fpr:0.495, fnr:0.495, precision:0.941, recall:0.505, F1:0.658
-bad(pred/true):  45/  95, fpr:0.547, fnr:0.547, precision:0.956, recall:0.453, F1:0.614
[Epoch: 1/40] train_loss: 0.415, train_acc: 0.815, val_loss: 0.403, val_acc: 0.862, Time Cost: 6.299 s
-bad(pred/true):  43/  95, fpr:0.558, fnr:0.558, precision:0.977, recall:0.442, F1:0.609
-bad(pred/true):  70/  95, fpr:0.337, fnr:0.337, precision:0.900, recall:0.663, F1:0.764
[Epoch: 2/40] train_loss: 0.360, train_acc: 0.866, val_loss: 0.349, val_acc: 0.867, Time Cost: 6.091 s
-bad(pred/true):  72/  95, fpr:0.253, fnr:0.253, precision:0.986, recall:0.747, F1:0.850
-bad(pred/true):  90/  95, fpr:0.147, fnr:0.147, precision:0.900, recall:0.853, F1:0.876
[Epoch: 3/40] train_loss: 0.307, train_acc: 0.923, val_loss: 0.301, val_acc: 0.921, Time Cost: 6.421 s
-bad(pred/true):  81/  95, fpr:0.158, fnr:0.158, precision:0.988, recall:0.842, F1:0.909
-bad(pred/true):  85/  95, fpr:0.147, fnr:0.147, precision:0.953, re

## test

In [51]:
# 超参数：
HIDDEN_DIM = 100  # 隐藏层维度 100
ENCODE_DIM = 128  # 编码层？维度 128，
LABELS = 2  # 标签种类数量
EPOCHS = 30
BATCH_SIZE = 300
USE_GPU = True

# 测试。
total_acc = total_loss = total = i = 0  # 重置标记
model = best_model  # 使用在验证集上 acc 最高的模型进行测试
metrics_index = ['FNR_', 'FNR_', 'precision_', 'recall_', 'F1_', 'acc_']
res = {k:[] for k in ['FNR_', 'FNR_', 'precision_', 'recall_', 'F1_', 'acc_']}

while i < len(test_data):
    batch = get_batch(test_data, i, BATCH_SIZE)
    i += BATCH_SIZE
    test_inputs, test_labels = batch
    if USE_GPU:
        test_inputs, test_labels = test_inputs, test_labels.cuda()

    model.batch_size = len(test_labels)
    model.hidden = model.init_hidden()
    output = model(test_inputs)

    loss = loss_function(output, Variable(test_labels))

    _, predicted = torch.max(output.data, 1)
    total_acc += (predicted == test_labels).sum()
    total += len(test_labels)
    total_loss += loss.item() * len(test_inputs)

    # 计算 precision、recall、accuracy、f1
    y_pred = predicted.tolist()
    y_true = test_labels.tolist()
    FPR_, recall_, thresholds = metrics.roc_curve(y_true, y_pred)
    FNR_ = 1 - recall_
    recall_ = metrics.recall_score(y_true, y_pred)
    precision_ = metrics.precision_score(y_true, y_pred)
    F1_ = metrics.f1_score(y_true, y_pred)
    acc_ = metrics.accuracy_score(y_true, y_pred)
    res['FNR_'] = FNR_
    res['FNR_'] = FNR_
    res['precision_'] = precision_
    res['recall_'] = recall_
    res['F1_'] = F1_
    res['acc_'] = acc_
    print('-bad(pred/true):%4d/%4d, fpr:%.4f, fnr:%.4f, precision:%.4f, recall:%.4f, F1:%.4f, acc:%.4f' 
      %(sum(y_pred), sum(y_true), FNR_[1], FNR_[1], precision_, recall_, F1_, acc_))

    
print("Testing results(Acc):", total_acc.item() / total)

-bad(pred/true):  87/  87, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):  85/  86, fpr:0.0116, fnr:0.0116, precision:1.0000, recall:0.9884, F1:0.9942, acc:0.9967
-bad(pred/true):  85/  85, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
-bad(pred/true):   4/   4, fpr:0.0000, fnr:0.0000, precision:1.0000, recall:1.0000, F1:1.0000, acc:1.0000
Testing results(Acc): 0.9989059080962801


## 分析

In [33]:
''' 将数据集保存成 csv，以人工分析，为什么准确率这么高

cwe119_c_all.csv 文件大小：2.93G。太大了
Index(['id', 'name', 'code', 'ast', 'label', 'corpus', 'blocks'], dtype='object')

train_data
val_data
test_data
'''
# import pandas as pd
cwe399_asts_df.columns

# 划分
cwe399_part = cwe399_asts_df[['id', 'name', 'label', 'blocks']]
cwe399_part.to_csv('data/cwe399_c_part.csv')



# 将 AST 按语句划分

In [ ]:
'''
    r: 传入的 FileAST 对象, 即一个程序的抽象语法树, bolcks 用于接收结果的列表对象
    递归调用.
    仅处理 ['FuncDef', 'If', 'For', 'While', 'DoWhile'] 和 'Compound' 这两种类型的节点,过滤掉了大量的 Typedef 节点
    调用 tree.ASTNode(), 
'''

In [1]:
# generate block sequences with index representations
def generate_block_seqs(self,data_path,part):
    from prepare_data import get_blocks as func
    from gensim.models.word2vec import Word2Vec

    word2vec = Word2Vec.load(self.root+'train/embedding/node_w2v_' + str(self.size)).wv
    vocab = word2vec.vocab
    max_token = word2vec.syn0.shape[0] # number of tokens

    def tree_to_index(node):
        token = node.token
        result = [vocab[token].index if token in vocab else max_token]
        children = node.children
        for child in children:
            result.append(tree_to_index(child))
        return result

    def trans2seq(r):
        blocks = []
        func(r, blocks)
        tree = []
        for b in blocks:
            btree = tree_to_index(b)
            tree.append(btree)
        return tree

    trees = pd.read_pickle(data_path)
    trees['code'] = trees['code'].apply(trans2seq)
    trees.to_pickle(self.root+part+'/blocks.pkl')

## 替换 AST 中每条语句的的 子树st-tree 为语句嵌入

In [3]:
'''

FileAST: 
  FuncDef: 
    Decl: hello, [], [], []
      FuncDecl: 
        ParamList: 
          ID: a
          ID: b
        TypeDecl: hello, []
          IdentifierType: ['int']
    Compound: 
      Decl: bufsize, [], [], []
        TypeDecl: bufsize, []
          IdentifierType: ['int']
      Decl: strbuf, [], [], []
        PtrDecl: []
          TypeDecl: strbuf, []
            IdentifierType: ['char']
      If: 
        BinaryOp: ==
          ID: a
          ID: b
        Compound: 
          Assignment: +=
            ID: a
            Constant: int, 1
      Return: 
        BinaryOp: +
          ID: a
          ID: b

FileAST: 
    FuncDef_em
    compound:
        Decl_bufsize
        Decl_strbuf
        IF
            IF_CONDICT
            IF_TRUE
                Assignment
        Return
'''
ast_plus = generate_block_seqs('AST_CWE119.pkl'):

"\n\nFileAST: \n  FuncDef: \n    Decl: hello, [], [], []\n      FuncDecl: \n        ParamList: \n          ID: a\n          ID: b\n        TypeDecl: hello, []\n          IdentifierType: ['int']\n    Compound: \n      Decl: bufsize, [], [], []\n        TypeDecl: bufsize, []\n          IdentifierType: ['int']\n      Decl: strbuf, [], [], []\n        PtrDecl: []\n          TypeDecl: strbuf, []\n            IdentifierType: ['char']\n      If: \n        BinaryOp: ==\n          ID: a\n          ID: b\n        Compound: \n          Assignment: +=\n            ID: a\n            Constant: int, 1\n      Return: \n        BinaryOp: +\n          ID: a\n          ID: b\n\nFileAST: \n    FuncDef_em\n    compound:\n        Decl_bufsize\n        Decl_strbuf\n        IF\n            IF_CONDICT\n            IF_TRUE\n                Assignment\n        Return\n"

In [2]:
class ASTNode(object):
    def __init__(self, node):
        self.node = node
        # self.vocab = word_map
        self.is_str = isinstance(self.node, str)
        self.token = self.get_token()
        # self.index = self.token_to_index(self.token)
        self.children = self.add_children()

    def is_leaf(self):
        if self.is_str:
            return True
        return len(self.node.children()) == 0

    def get_token(self, lower=True):
        if self.is_str:
            return self.node
        name = self.node.__class__.__name__
        token = name
        is_name = False
        if self.is_leaf():
            attr_names = self.node.attr_names
            if attr_names:
                if 'names' in attr_names:
                    token = self.node.names[0]
                elif 'name' in attr_names:
                    token = self.node.name
                    is_name = True
                else:
                    token = self.node.value
            else:
                token = name
        else:
            if name == 'TypeDecl':
                token = self.node.declname
            if self.node.attr_names:
                attr_names = self.node.attr_names
                if 'op' in attr_names:
                    if self.node.op[0] == 'p':
                        token = self.node.op[1:]
                    else:
                        token = self.node.op
        if token is None:
            token = name
        if lower and is_name:
            token = token.lower()
        return token

    # def token_to_index(self, token):
    #     self.index = self.vocab[token].index if token in self.vocab else MAX_TOKENS
    #     return self.index

    # def get_index(self):
    #     return self.index

    def add_children(self):
        if self.is_str:
            return []
        children = self.node.children()
        if self.token in ['FuncDef', 'If', 'While', 'DoWhile','Switch']:
            return [ASTNode(children[0][1])]
        elif self.token == 'For':
            return [ASTNode(children[c][1]) for c in range(0, len(children)-1)]
        else:
            return [ASTNode(child) for _, child in children]

    def children(self):
        return self.children
    #     if self.is_str:
    #         return []
    #     return [ASTNode(child) for _, child in self.node.children() if child.]

# Tensorflow

In [2]:
test_model('model/model_cwe399_cleaned', 'vec/fast_algorithms_trees_tf_rm.pkl', 'vec/vec_cwe399_cleaned.pkl')


load file ...
init network ...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from model/model_cwe399_cleaned/cnn_tree.ckpt-4200
('Accuracy:', 0.375)
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         1
        good       0.86      0.40      0.55        15

   micro avg       0.38      0.38      0.38        16
   macro avg       0.43      0.20      0.27        16
weighted avg       0.80      0.38      0.51        16

[[0 1]
 [9 6]]
('correct_labels: ', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1])
('predictions: ', [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0])


# end